In [ ]:
import ipyvuetify as v
from ipyleaflet import Map, Marker, Icon
import uuid, hashlib, json, os
from pathlib import Path
import pandas as pd
import base64
from IPython.display import display

# --- Symbol options and SVGs (abbreviated for brevity) ---
symbol_options = [
    'Friendly Infantry', 'Hostile Infantry', 'Neutral Infantry', 'Unknown Infantry',
    'Friendly Armor', 'Hostile Armor', 'Friendly Artillery', 'Hostile Artillery',
    'Friendly Engineer', 'Friendly Signal', 'Friendly Medical',
    'Friendly Recon', 'Friendly SF', 'Friendly HQ',
    'Friendly CBRN', 'Friendly UAV', 'Friendly Radar',
    'Friendly INT', 'Hostile INT', 'Neutral INT', 'Unknown INT',
    'Friendly SIGINT', 'Friendly ELINT',
    'Friendly Air', 'Hostile Air', 'Neutral Air',
    'Friendly Naval', 'Hostile Naval', 'Neutral Naval'
]
SVGs = {
    'Friendly Infantry': "<svg xmlns=\"http://www.w3.org/2000/svg\" width=\"80\" height=\"60\"><rect width=\"80\" height=\"60\" fill=\"blue\"/><g stroke=\"white\" stroke-width=\"4\" stroke-linecap=\"round\"><line x1=\"25\" y1=\"45\" x2=\"55\" y2=\"15\"/><line x1=\"55\" y1=\"45\" x2=\"25\" y2=\"15\"/></g></svg>",
    # ... (add more SVGs as needed) ...
}
def svg_to_datauri(svg):
    svg_b64 = base64.b64encode(svg.encode('utf-8')).decode('utf-8')
    return f'data:image/svg+xml;base64,{svg_b64}'

def wrap_with_red_ring(inner_svg: str) -> str:
    return f"""
<svg xmlns='http://www.w3.org/2000/svg' width='90' height='70' viewBox='0 0 90 70'>
  <circle cx='45' cy='35' r='30' fill='none' stroke='#dc2626' stroke-width='4'/>
  <g transform='translate(5,5)'>
    {inner_svg}
  </g>
</svg>
"""

def wrap_with_red_dot(base_svg: str, r: int = 12, cx: int = 16, cy: int = 16) -> str:
    return f"""
<svg xmlns='http://www.w3.org/2000/svg' width='90' height='70' viewBox='0 0 90 70'>
  <g transform='translate(5,5)'>
    {base_svg}
  </g>
  <circle cx='{cx}' cy='{cy}' r='{r}' fill='#dc2626'/>
</svg>
"""

# --- State ---
selected_symbol = symbol_options[0]
current_symbol = [selected_symbol]  # Use a mutable object to allow updates in closure
log_df = pd.DataFrame(columns=['symbol', 'when', 'coords', 'reported_at', 'reporter_id', 'reporter_type', 'source_reliability', 'information_credibility', 'description', 'source_reference'])

# --- Symbol selector ---
symbol_select = v.Select(items=symbol_options, v_model=selected_symbol, label='Select Symbol', outlined=True, style_='max-width: 300px;')
def on_symbol_change(widget, event, data):
    current_symbol[0] = data
symbol_select.on_event('change', on_symbol_change)

# --- Map setup ---
m = Map(center=(51.5, -0.12), zoom=12)
m.layout.width = '900px'
m.layout.height = '500px'
marker_objects = []
def add_marker(lat, lng, symbol):
    icon_url = svg_to_datauri(SVGs.get(symbol, SVGs[symbol_options[0]]))
    icon = Icon(icon_url=icon_url, icon_size=[40, 30], icon_anchor=[20, 15])
    marker = Marker(location=(lat, lng), icon=icon)
    m.add_layer(marker)
    marker_objects.append(marker)

def on_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        latlng = kwargs['coordinates']
        add_marker(latlng[0], latlng[1], current_symbol[0])
# m.on_interaction(on_map_click)  # disabled to avoid duplicate legacy map behavior

# --- Vuetify Tabs Layout ---
tabs = v.Tabs(v_model=0, children=[
    v.Tab(children=['Map']),
    v.Tab(children=['Log Table'])
])
tab_items = v.TabsItems(v_model=0, children=[
    v.TabItem(children=[
        v.Container(children=[
            v.Row(children=[symbol_select]),
            v.Row(children=[m])
        ])
    ]),
    v.TabItem(children=[
        v.Container(children=[
            v.Row(children=[v.Html(tag='div', children=['Log Table coming soon...'], style_='font-size: 20px; color: #888; margin: 40px;')])
        ])
    ])
])
app = v.Container(children=[
    v.Toolbar(children=[v.ToolbarTitle(children=['INTCELL Dashboard'])]),
    tabs,
    tab_items
], style_='max-width: 100vw;')
# display(app)  # disabled to avoid duplicate legacy UI



In [3]:
# ict-irt mapping and logging app.

import ipywidgets as widgets
from ipyleaflet import Map, Marker, Icon, basemaps, basemap_to_tiles, LayersControl, FullScreenControl, CircleMarker
from IPython.display import display, clear_output, HTML
import html as _html
from typing import Any
import time
import base64
import pandas as pd
from datetime import datetime, timedelta
import ipyvuetify as v
import math

# ---- removing border from all ipywidgets dropdowns ----
display(HTML("""
<style>
/* Remove border from all ipywidgets dropdowns */
.widget-dropdown select {
    border: none !important;
    box-shadow: none !important;
    background-color: transparent !important;
}
/* Optional: Remove border from the dropdown container */
.widget-dropdown {
    border: none !important;
    box-shadow: none !important;
    background-color: transparent !important;
}
</style>
"""))

# Ensure dialogs are above Leaflet fullscreen overlays and align toolbar content
display(HTML("""
<style>
.v-application .v-overlay__content { z-index: 99999 !important; }
.v-application .v-overlay__scrim { z-index: 99998 !important; }
.ict-toolbar .v-input { margin-bottom: 0 !important; }
.ict-symbol-preview-wrap { display:flex; align-items:center; margin-top: -6px; }
</style>
"""))

# ---- Data + SVGs ----
symbol_options = [
    "Friendly Infantry", "Hostile Infantry", "Neutral Infantry", "Unknown Infantry",
    "Friendly Armor", "Hostile Armor", "Friendly Artillery", "Hostile Artillery",
    "Friendly Engineer", "Friendly Signal", "Friendly Medical",
    "Friendly Recon", "Friendly SF", "Friendly HQ",
    "Friendly CBRN", "Friendly UAV", "Friendly Radar",
    "Friendly INT", "Hostile INT", "Neutral INT", "Unknown INT",
    "Friendly SIGINT", "Friendly ELINT",
    "Friendly Air", "Hostile Air", "Neutral Air",
    "Friendly Naval", "Hostile Naval", "Neutral Naval"
]

from svgs import SVGs

def svg_to_datauri(svg):
    svg_b64 = base64.b64encode(svg.encode('utf-8')).decode('utf-8')
    return f'data:image/svg+xml;base64,{svg_b64}'
def now_str(): return datetime.now().strftime("%Y-%m-%d:%H:%M:%S")
def dt_from_str(s): return datetime.strptime(s, "%Y-%m-%d:%H:%M:%S")
def icon_size_for_zoom(zoom):
    ZOOM_ICON_TABLE = {7:[12,9],8:[16,12],9:[20,15],10:[26,19],11:[32,24],12:[40,30],13:[48,36],14:[32,24],15:[24,18],16:[16,12],17:[10,8],18:[7,5]}
    z = int(round(zoom))
    sizes = sorted(ZOOM_ICON_TABLE.items())
    for k, sz in reversed(sizes):
        if z >= k:
            # Max zoom: make icons large and readable
            if z >= 18:
                return [80, 60]
            # Very high zoom (just under max)
            if z >= 17:
                return [64, 48]
            # At high zoom levels (15-16), increase by 30%
            if z >= 15:
                return [int(round(sz[0] * 1.3)), int(round(sz[1] * 1.3))]
            return sz
    return sizes[0][1]

# ---- DataFrame Schema (all columns) ----
log_columns = [
    # Identity / basic map icon linkage
    "symbol", "affiliation", "unit_identity", "report_type", "thematic_tags",
    # Entry identity
    "entry_id",
    # Time
    "when", "reported_at", "staleness_minutes",
    # Location
    "coords", "lat", "lon", "mgrs", "mgrs_precision_m", "geo_source", "location_text",
    # Contact (entity-level)
    "contact_uuid", "contact_ref", "contact_short", "contact_source_id",
    "contact_status", "contact_close_reason", "contact_close_dtg",
    # Content
    "size", "activity", "equipment", "time_observed", "description", "attachments_ref",
    # Evaluation
    "source_reliability", "information_credibility", "classification", "handling_instructions",
    "analyst_assessment", "analyst_confidence",
    # Relevance / Tasking
    "pir_id", "sir_id", "ccir_flag", "priority", "critical_int", "critical_manual",
    # Provenance
    "reporting_unit", "reporter_type", "reporter_id", "collection_method",
    # Workflow / QA
    "status", "action_taken", "dissemination_list", "duplicates_of",
    "created_by", "reviewed_by", "last_updated_dtg",
    # UIDs
    "ict_uid", "short_uid", "source_uid",
    # Legacy / misc
    "source_reference"
]
log_df = pd.DataFrame(columns=log_columns)

# ---- Versions (append-only history) ----
# __file__ is not defined under some notebook/Voila contexts; fall back to this file's known workspace path
try:
    _base_dir = Path(os.path.dirname(__file__))
except NameError:
    # Use the project path provided by the workspace
     _base_dir = Path(os.getcwd()).parent()
# Versions file now directly inside /app
VERSIONS_FILE = _base_dir / 'versions.csv'
versions_columns = [
    'entry_id','rev','rev_parent','is_current','op','rev_time','rev_user','row_hash','payload_json'
]
try:
    versions_df = pd.read_csv(VERSIONS_FILE) if VERSIONS_FILE.exists() else pd.DataFrame(columns=versions_columns)
except Exception:
    versions_df = pd.DataFrame(columns=versions_columns)

def _save_versions():
    try:
        versions_df.to_csv(VERSIONS_FILE, index=False)
    except Exception:
        pass

def _canonical_payload(row: pd.Series) -> dict:
    payload = {k: row.get(k, '') for k in log_columns}
    return payload

def _row_fingerprint(payload: dict) -> str:
    """Create a stable fingerprint for de-duplication.
    Hash the entire payload so any field change (e.g., Critical Int) creates a new fingerprint.
    """
    data = json.dumps(payload, sort_keys=True, separators=(',', ':'), default=str)
    return hashlib.sha1(data.encode('utf-8')).hexdigest()

def _append_version(entry_id: str, op: str, payload: dict, user: str = ''):
    global versions_df
    try:
        subset = versions_df[versions_df['entry_id'] == entry_id]
        last_rev = int(subset['rev'].max()) if len(subset) else 0
        new_rev = last_rev + 1
        # compute fingerprint for duplicate detection
        new_hash = _row_fingerprint(payload)
        # If there is a current version and the content hash is identical, skip no-op updates
        try:
            cur = subset[subset['is_current'] == True]
            if len(cur) > 0:
                cur_hash = str(cur.iloc[-1]['row_hash'])
                if cur_hash == new_hash and op != 'revert':
                    return
        except Exception:
            pass
        # mark previous current as false
        if len(subset):
            versions_df.loc[versions_df['entry_id'] == entry_id, 'is_current'] = False
        row = {
            'entry_id': entry_id,
            'rev': new_rev,
            'rev_parent': last_rev if last_rev else 0,
            'is_current': True,
            'op': op,
            'rev_time': datetime.utcnow().strftime('%Y-%m-%d:%H:%M:%S'),
            'rev_user': user or '',
            'row_hash': new_hash,
            'payload_json': json.dumps(payload, separators=(',', ':'), default=str),
        }
        versions_df = pd.concat([versions_df, pd.DataFrame([row])], ignore_index=True)
        _save_versions()
    except Exception:
        pass

# ---- History Dialog (simple) ----
_history_entry_id_current = ['']
_history_items = []  # list of (rev, label)
history_dialog = v.Dialog(v_model=False, max_width='720px')
history_title = v.CardTitle(children=['Revision History'])
history_list = v.Select(items=[], v_model=None, dense=True, filled=True, label='Select revision', class_='nato-input')
history_show_only_changed = v.Switch(v_model=True, label='Show only changed fields', class_='ma-2')
history_info = v.Html(tag='div', children=[''])
# Use ipywidgets.HTML to ensure HTML content renders reliably inside the dialog
history_diff_html = widgets.HTML(value="", layout=widgets.Layout(overflow_y='auto', overflow_x='hidden', max_height='40vh', width='100%'))
history_revert_btn = v.Btn(children=['Revert to selected'], class_='ict-btn ict-save ma-2')
history_close_btn = v.Btn(children=['Close'], class_='ict-btn ict-cancel ma-2')
history_card = v.Card(children=[history_title, v.CardText(children=[history_list, history_show_only_changed, history_info, history_diff_html]), v.CardActions(children=[v.Spacer(), history_close_btn, history_revert_btn])])
history_dialog.children = [history_card]

def open_history_dialog(entry_id: str):
    global _history_items
    try:
        _history_entry_id_current[0] = str(entry_id)
        rows = versions_df[versions_df['entry_id'] == entry_id].sort_values('rev', ascending=False)
        _history_items = []
        items = []
        current_rev = None
        for _, r in rows.iterrows():
            label = f"rev {int(r['rev'])} • {r.get('op','')} • {r.get('rev_time','')}"
            items.append({'text': label, 'value': int(r['rev'])})
            _history_items.append((int(r['rev']), r))
            if bool(r.get('is_current', False)):
                current_rev = int(r['rev'])
        history_list.items = items
        # Preselect the current revision if known; otherwise default to latest
        if current_rev is not None:
            history_list.v_model = current_rev
        else:
            history_list.v_model = items[0]['value'] if items else None
        history_info.children = [f"{len(items)} revisions"]
        try:
            _on_history_select(None, None, history_list.v_model)
        except Exception:
            pass
        history_dialog.v_model = True
    except Exception as e:
        history_info.children = [f"Error loading history: {e}"]
        history_dialog.v_model = True

def _revert_to_selected(widget=None, *args):
    global log_df, versions_df
    try:
        entry_id = _history_entry_id_current[0]
        rev_sel = history_list.v_model
        if not entry_id or rev_sel is None:
            history_info.children = ['Select a revision first']
            return
        row = versions_df[(versions_df['entry_id'] == entry_id) & (versions_df['rev'] == int(rev_sel))]
        if row.empty:
            history_info.children = ['Revision not found']
            return
        payload = json.loads(row.iloc[0]['payload_json'])
        # Find current row index by entry_id
        try:
            idx = int(log_df[log_df['entry_id'] == entry_id].index[0])
        except Exception:
            history_info.children = ['Entry not found in current table']
            return
        # Apply payload to DF
        for k in log_columns:
            try:
                log_df.at[idx, k] = payload.get(k, '')
            except Exception:
                pass
        # Mark selected revision as current without creating a new one
        try:
            versions_df.loc[versions_df['entry_id'] == entry_id, 'is_current'] = False
            versions_df.loc[(versions_df['entry_id'] == entry_id) & (versions_df['rev'] == int(rev_sel)), 'is_current'] = True
            _save_versions()
            # Keep the selection on the revision just made current
            try:
                history_list.v_model = int(rev_sel)
            except Exception:
                pass
        except Exception:
            pass
        history_info.children = [f"Reverted to rev {rev_sel} (no new revision)"]
        # If the modal is open for this row, re-open it with the reverted payload and refresh the title
        try:
            if entry_dialog.v_model and (_dlg_row_index[0] == idx):
                history_dialog.v_model = False
                _open_entry_dialog(payload, idx)
                _refresh_modal_title()
            else:
                refresh_all(refresh_map_flag=True)
                history_dialog.v_model = False
        except Exception:
            refresh_all(refresh_map_flag=True)
            history_dialog.v_model = False
    except Exception as e:
        history_info.children = [f"Revert failed: {e}"]

history_revert_btn.on_event('click', _revert_to_selected)
history_close_btn.on_event('click', lambda *_: setattr(history_dialog, 'v_model', False))

def _on_history_select(widget, event, data):
    try:
        entry_id = _history_entry_id_current[0]
        rev_sel = data if data is not None else history_list.v_model
        if not entry_id or rev_sel is None:
            history_diff_html.value = ''
            return
        # selected payload
        row_sel = versions_df[(versions_df['entry_id'] == entry_id) & (versions_df['rev'] == int(rev_sel))]
        if row_sel.empty:
            history_diff_html.value = ''
            return
        payload_sel = json.loads(row_sel.iloc[0]['payload_json'])
        # current payload
        try:
            idx = int(log_df[log_df['entry_id'] == entry_id].index[0])
            payload_cur = _canonical_payload(log_df.loc[idx])
        except Exception:
            payload_cur = {}
        # Section grouping map
        section_map = {
            'Identity': ["symbol","affiliation","unit_identity","report_type","thematic_tags"],
            'Time': ["when","reported_at","staleness_minutes"],
            'Location': ["coords","lat","lon","mgrs","mgrs_precision_m","geo_source","location_text"],
            'Content (SALUTE)': ["size","activity","equipment","time_observed","description","attachments_ref"],
            'Evaluation (5x5x5)': ["source_reliability","information_credibility","classification","handling_instructions","analyst_assessment","analyst_confidence"],
            'Relevance / Tasking': ["pir_id","sir_id","ccir_flag","priority","critical_int","critical_manual"],
            'Provenance': ["reporting_unit","reporter_type","reporter_id","collection_method"],
            'Workflow / QA': ["status","action_taken","dissemination_list","duplicates_of","created_by","reviewed_by","last_updated_dtg","ict_uid","short_uid","source_uid"],
            'Legacy / misc': ["source_reference"],
        }
        # Build grouped diff HTML with 2-column grid (Current | Selected)
        def _fmt_val(v: object) -> str:
            try:
                s = '' if v is None else str(v)
            except Exception:
                s = ''
            if s == '':
                return '<span style="color:#6b7280">(empty)</span>'
            low = s.strip().lower()
            if low in ('true','false'):
                if low == 'true':
                    return '<span style="display:inline-block;padding:2px 8px;border-radius:999px;background:#10b981;color:#fff;font-weight:600;">True</span>'
                else:
                    return '<span style="display:inline-block;padding:2px 8px;border-radius:999px;border:1px solid #9ca3af;color:#374151;background:#fff;">False</span>'
            return _html.escape(s)

        only_changed = bool(history_show_only_changed.v_model)
        sections_out = []
        for section, fields in section_map.items():
            rows = []
            for k in fields:
                v_cur = payload_cur.get(k, '')
                v_sel = payload_sel.get(k, '')
                changed = str(v_cur) != str(v_sel)
                if only_changed and not changed:
                    continue
                field_name = _html.escape(k)
                cur_html = _fmt_val(v_cur)
                sel_html = _fmt_val(v_sel)
                row_bg = '#fff7ed' if changed else '#ffffff'
                rows.append(
                    "<tr style='background:%s'><td style='padding:6px 8px;vertical-align:top;font-weight:600;width:34%%'>%s</td><td style='padding:6px 8px;vertical-align:top;width:33%%;word-break:break-word;'>%s</td><td style='padding:6px 8px;vertical-align:top;width:33%%;word-break:break-word;'>%s</td></tr>" % (row_bg, field_name, cur_html, sel_html)
                )
            if rows:
                table = (
                    "<div style='margin:10px 0 14px 0;'>"
                    + "<div style='font-weight:800;margin:6px 0;'>%s</div>" % _html.escape(section)
                    + "<table style='width:100%%;border-collapse:collapse;font-size:14px'>"
                    + "<thead><tr>"
                    + "<th style='text-align:left;padding:6px 8px;border-bottom:1px solid #e5e7eb;color:#374151;width:34%%'>Field</th>"
                    + "<th style='text-align:left;padding:6px 8px;border-bottom:1px solid #e5e7eb;color:#374151;width:33%%'>Current</th>"
                    + "<th style='text-align:left;padding:6px 8px;border-bottom:1px solid #e5e7eb;color:#374151;width:33%%'>Selected</th>"
                    + "</tr></thead><tbody>"
                    + "".join(rows)
                    + "</tbody></table></div>"
                )
                sections_out.append(table)

        if not sections_out:
            html = "<div style='color:#374151; font-family: -apple-system, BlinkMacSystemFont, Segoe UI, Roboto, Helvetica, Arial, sans-serif; font-size:14px;'>No differences from current</div>"
        else:
            content = "<div style='font-family: -apple-system, BlinkMacSystemFont, Segoe UI, Roboto, Helvetica, Arial, sans-serif; font-size:14px; line-height:1.35'>" + "".join(sections_out) + "</div>"
            html = "<div style='max-height:40vh; overflow-y:auto; padding-right:6px;'>" + content + "</div>"
        # Render diff
        history_diff_html.value = html
    except Exception as e:
        history_diff_html.value = f"<div style='color:#b91c1c'>Diff error: {e}</div>"

history_list.on_event('change', _on_history_select)
try:
    history_show_only_changed.observe(lambda c: _on_history_select(None, None, history_list.v_model), names=['v_model'])
except Exception:
    pass

# --- UI State ---
sort_col, sort_desc = 'reported_at', True
date_filter_mode, date_filter_col = 'all', 'when'
date_filter_from, date_filter_to = '', ''
page_size, page_num = 10, 0
selected_symbols = set(symbol_options)
editing_row_idx = [None]
creating_row = [False]
editing_row_id = [-1]
when_range_state = [[None, None]]  # epoch seconds [from, to]
reported_range_state = [[None, None]]
UNIT_CODE = 'ICT'
# Cache the highest ICT sequence we have generated per date to avoid duplicates
_ict_uid_seq_cache = {}

def generate_ict_uid(unit_code: str = UNIT_CODE) -> str:
    today = datetime.now().strftime('%Y%m%d')
    prefix = f"{unit_code}-{today}-"
    # Use a simple monotonic cache per date to guarantee increment-on-call
    try:
        max_seq = int(_ict_uid_seq_cache.get(today, 0))
    except Exception:
        max_seq = 0
    try:
        # Bootstrap cache from DataFrame the first time for this date
        if max_seq == 0:
            existing = []
            for val in list(log_df.get('ict_uid', []) or []):
                if isinstance(val, str) and val.startswith(prefix):
                    try:
                        existing.append(int(val.split('-')[-1]))
                    except Exception:
                        pass
            if existing:
                max_seq = max(existing)
        # Also consider the value currently displayed in the modal (if any)
        try:
            current_modal_ict = str(ict_uid_tf.v_model) if 'ict_uid_tf' in globals() else ''
            if current_modal_ict and current_modal_ict.startswith(prefix):
                try:
                    max_seq = max(max_seq, int(current_modal_ict.split('-')[-1]))
                except Exception:
                    pass
        except Exception:
            pass
    except Exception:
        pass
    # Increment cache and return next
    next_seq = max_seq + 1
    _ict_uid_seq_cache[today] = next_seq
    return f"{prefix}{next_seq:03d}"

def derive_short_uid(ict_uid: str) -> str:
    try:
        parts = ict_uid.split('-')
        yyyymmdd, seq = parts[1], parts[2]
        ddmm = yyyymmdd[6:8] + yyyymmdd[4:6]
        return f"{seq}-{ddmm}"
    except Exception:
        return ''

def generate_contact_ref() -> str:
    today = datetime.now().strftime('%Y%m%d')
    prefix = f"CT-{today}-"
    try:
        existing = []
        for val in list(log_df.get('contact_ref', []) or []):
            if isinstance(val, str) and val.startswith(prefix):
                try:
                    existing.append(int(val.split('-')[-1]))
                except Exception:
                    pass
        next_seq = (max(existing) + 1) if existing else 1
    except Exception:
        next_seq = 1
    return f"{prefix}{next_seq:03d}"

def derive_contact_short(contact_ref: str) -> str:
    try:
        parts = contact_ref.split('-')
        if len(parts) >= 3:
            yyyymmdd, seq = parts[1], parts[2]
            ddmm = yyyymmdd[6:8] + yyyymmdd[4:6]
            return f"CT{seq}-{ddmm}"
    except Exception:
        pass
    return ''

edit_mode = widgets.ToggleButton(
    value=False, description="Edit Table", icon="edit", button_style="info",
    layout=widgets.Layout(width="120px", min_height="38px", max_height="38px", margin="0 0 0 14px")
)

symbol_dropdown = widgets.Dropdown(options=symbol_options, value="Friendly Infantry", description="Symbol:")
#add_info = widgets.HTML(value="<b>Click map to add marker</b>", layout=widgets.Layout(margin="0 0 10px 0"))

# ---- Symbol Multi-Select for Filtering ----
symbol_select_dropdown_btn = widgets.ToggleButton(value=False, icon="filter", description="Symbols", layout=widgets.Layout(width="120px", height="32px", min_width="120px"))
symbol_select_box = widgets.SelectMultiple(options=symbol_options, value=tuple(symbol_options), rows=10, layout=widgets.Layout(width="210px", margin="2px 0 0 0", max_height="220px"))
symbol_dropdown_outer = widgets.Box(layout=widgets.Layout(position='relative', min_width="120px", max_width="140px"))
symbol_dropdown_inner = widgets.VBox([symbol_select_dropdown_btn], layout=widgets.Layout(min_width="120px", max_width="140px"))
symbol_dropdown_outer.children = [symbol_dropdown_inner]
def update_symbol_dropdown_visibility(*a):
    if symbol_select_dropdown_btn.value:
        symbol_dropdown_inner.children = [symbol_select_dropdown_btn, symbol_select_box]
    else:
        symbol_dropdown_inner.children = [symbol_select_dropdown_btn]
symbol_select_dropdown_btn.observe(update_symbol_dropdown_visibility, names='value')
def on_symbol_filter_change(change):
    global selected_symbols, page_num
    selected_symbols = set(change['new'])
    page_num = 0
    refresh_all(refresh_map_flag=False)
symbol_select_box.observe(on_symbol_filter_change, names='value')

# ---- Date Filtering Controls ----
date_filter_dropdown = widgets.Dropdown(
    options=[('All', 'all'), ('Last 1h', '1h'), ('Last 24h', '24h'), ('Custom', 'custom')],
    value='all', description='Date:', layout=widgets.Layout(width="165px", min_width="120px", max_width="180px")) #, border="1px solid #444"
date_col_dropdown = widgets.Dropdown(
    options=[('When', 'when'), ('Reported At', 'reported_at')],
    value='when', description='On:', layout=widgets.Layout(width="165px", min_width="120px", max_width="180px")) #, border="1px solid #444"
date_from_picker = widgets.Text(
    value='', placeholder='YYYY-MM-DD:HH:MM:SS', description='From:', layout=widgets.Layout(width="160px")
)
date_to_picker = widgets.Text(
    value='', placeholder='YYYY-MM-DD:HH:MM:SS', description='To:', layout=widgets.Layout(width="160px")
)
def on_date_filter_change(change):
    global date_filter_mode, page_num
    date_filter_mode = change['new']
    page_num = 0
    refresh_all(refresh_map_flag=False)
date_filter_dropdown.observe(on_date_filter_change, names='value')
def on_date_col_change(change):
    global date_filter_col
    date_filter_col = change['new']
    refresh_all(refresh_map_flag=False)
date_col_dropdown.observe(on_date_col_change, names='value')
def on_date_from_change(change):
    global date_filter_from
    date_filter_from = change['new']
    refresh_all(refresh_map_flag=False)
date_from_picker.observe(on_date_from_change, names='value')
def on_date_to_change(change):
    global date_filter_to
    date_filter_to = change['new']
    refresh_all(refresh_map_flag=False)
date_to_picker.observe(on_date_to_change, names='value')
def update_date_picker_visibility(*a):
    show = (date_filter_dropdown.value == 'custom')
    date_from_picker.layout.display = 'block' if show else 'none'
    date_to_picker.layout.display = 'block' if show else 'none'
update_date_picker_visibility()
date_filter_dropdown.observe(update_date_picker_visibility, names='value')

# ---- Filter Row Layout ----
def build_filter_row():
    symbol_area = widgets.Box([symbol_dropdown_outer], layout=widgets.Layout(min_width="125px", max_width="145px", margin="0 14px 0 0"))
    date_area = widgets.Box([date_filter_dropdown], layout=widgets.Layout(min_width="165px", max_width="185px", margin="0 14px 0 0"))
    datecol_area = widgets.Box([date_col_dropdown], layout=widgets.Layout(min_width="165px", max_width="185px", margin="0 14px 0 0"))
    date_range_area = widgets.HBox([date_from_picker, date_to_picker], layout=widgets.Layout(align_items="center", min_width="335px", max_width="355px"))
    row = widgets.HBox(
        [symbol_area, date_area, datecol_area, date_range_area],
        layout=widgets.Layout(width="99%", min_width="800px", align_items="center", flex_wrap="nowrap")
    )
    return row

# ---- Contact Activity Filter (replaces serial sliders) ----
def _epoch(dt_obj: datetime) -> int:
    try:
        return int(dt_obj.timestamp())
    except Exception:
        return 0

def _compute_time_bounds():
    # Determine global min/max from log_df 'when' and 'reported_at'
    try:
        times = []
        for col in ('when', 'reported_at'):
            for v in log_df.get(col, []):
                try:
                    if pd.notnull(v) and v:
                        times.append(_epoch(dt_from_str(v)))
                except Exception:
                    pass
        if not times:
            now_ = datetime.now()
            return _epoch(now_ - timedelta(hours=24)), _epoch(now_ + timedelta(hours=1))
        return min(times), max(times)
    except Exception:
        now_ = datetime.now()
        return _epoch(now_ - timedelta(hours=24)), _epoch(now_ + timedelta(hours=1))

time_min_epoch, time_max_epoch = _compute_time_bounds()

def _fmt_epoch(e: int) -> str:
    try:
        return datetime.fromtimestamp(int(e)).strftime('%Y-%m-%d:%H:%M:%S')
    except Exception:
        return ''

def _set_contact_activity_summary(text: str):
    try:
        contact_activity_label.children = [f"Contacts with last activity: {text}"]
    except Exception:
        pass

def _parse_dt(s: str):
    try:
        return dt_from_str(s)
    except Exception:
        return None

contact_activity_state = {'mode': 'Last 24h', 'from': None, 'to': None, 'include_closed': False}

def _apply_contact_activity_state():
    # Update global states used by filtered_sorted_df
    mode = contact_activity_state['mode']
    if mode == 'All':
        when_range_state[0] = [None, None]
    elif mode == 'Last 1h':
        now_ = datetime.now()
        when_range_state[0] = [_epoch(now_ - timedelta(hours=1)), _epoch(now_)]
    elif mode == 'Last 24h':
        now_ = datetime.now()
        when_range_state[0] = [_epoch(now_ - timedelta(hours=24)), _epoch(now_)]
    elif mode == 'Last 7d':
        now_ = datetime.now()
        when_range_state[0] = [_epoch(now_ - timedelta(days=7)), _epoch(now_)]
    elif mode == 'Custom…':
        f = _epoch(_parse_dt(contact_activity_state['from'])) if contact_activity_state['from'] else None
        t = _epoch(_parse_dt(contact_activity_state['to'])) if contact_activity_state['to'] else None
        when_range_state[0] = [f, t]

def _on_activity_mode_change(widget=None, event=None, data=None):
    contact_activity_state['mode'] = data or (contact_activity_select.v_model if 'contact_activity_select' in globals() else 'Last 24h')
    _apply_contact_activity_state()
    _set_contact_activity_summary(contact_activity_state['mode'])
    refresh_all(refresh_map_flag=True)

def _on_custom_from_change(widget=None, event=None, data=None):
    contact_activity_state['from'] = data
    if contact_activity_state['mode'] == 'Custom…':
        _apply_contact_activity_state()
        _set_contact_activity_summary('Custom')
        refresh_all(refresh_map_flag=True)

def _on_custom_to_change(widget=None, event=None, data=None):
    contact_activity_state['to'] = data
    if contact_activity_state['mode'] == 'Custom…':
        _apply_contact_activity_state()
        _set_contact_activity_summary('Custom')
        refresh_all(refresh_map_flag=True)

def _on_include_closed_change(widget=None, event=None, data=None):
    try:
        contact_activity_state['include_closed'] = bool(include_closed_checkbox.v_model)
    except Exception:
        contact_activity_state['include_closed'] = False
    refresh_all(refresh_map_flag=True)

# Bind when widgets are present (they are created in the toolbar)
def _bind_contact_filters():
    try:
        contact_activity_select.on_event('change', _on_activity_mode_change)
    except Exception:
        pass
    try:
        contact_from_picker.on_event('change', _on_custom_from_change)
    except Exception:
        pass
    try:
        contact_to_picker.on_event('change', _on_custom_to_change)
    except Exception:
        pass
    try:
        include_closed_checkbox.on_event('change', _on_include_closed_change)
    except Exception:
        pass

_bind_contact_filters()
_apply_contact_activity_state()

# ---- Filtering, Sorting, Pagination ----
def filtered_sorted_df():
    df = log_df.copy()
    # Safety: ensure essential columns exist; if empty, return the raw df to avoid accidental drop-all
    if 'symbol' not in df.columns:
        return df
    if selected_symbols:
        df = df[df["symbol"].isin(selected_symbols)]
    # Contact-centric Activity filter: compute last activity per contact_uuid (max 'when')
    try:
        wf, wt = when_range_state[0]
        if wf is not None or wt is not None:
            # Build per-contact last activity epoch (max of event_dtg 'when' and 'reported_at')
            def _row_epoch_two(row):
                try:
                    v1 = row.get('when')
                    v2 = row.get('reported_at')
                    e1 = _epoch(dt_from_str(v1)) if (pd.notnull(v1) and v1) else None
                    e2 = _epoch(dt_from_str(v2)) if (pd.notnull(v2) and v2) else None
                    if e1 is None and e2 is None:
                        return None
                    if e1 is None: return e2
                    if e2 is None: return e1
                    return max(e1, e2)
                except Exception:
                    return None
            needed_cols = [c for c in ['contact_uuid','when','reported_at'] if c in df.columns]
            if len(needed_cols) < 1:
                needed_cols = []
            tmp = df[needed_cols].copy() if needed_cols else pd.DataFrame()
            tmp['last_epoch'] = tmp.apply(_row_epoch_two, axis=1)
            if 'contact_uuid' in tmp.columns:
                grp = tmp.groupby('contact_uuid')['last_epoch'].max().dropna()
            else:
                grp = pd.Series(dtype='float64')
            # If we somehow have no epochs at all, do not filter anything
            if not grp.empty:
                allowed = set()
                for cu, ep in grp.items():
                    if (wf is None or ep >= wf) and (wt is None or ep <= wt):
                        allowed.add(cu)
                if 'contact_uuid' in df.columns:
                    df = df[df['contact_uuid'].isin(allowed)]
    except Exception:
        pass
    # Include/Exclude closed contacts based on checkbox
    try:
        include_closed = contact_activity_state.get('include_closed', False)
        if not include_closed:
            df = df[df['contact_status'].apply(lambda s: str(s).strip().lower() != 'closed')]
    except Exception:
        pass
    if date_filter_mode != 'all':
        col = date_filter_col
        now = datetime.now()
        if date_filter_mode == '1h':
            cutoff = now - timedelta(hours=1)
            df = df[df[col].apply(lambda d: dt_from_str(d) >= cutoff if pd.notnull(d) and d else False)]
        elif date_filter_mode == '24h':
            cutoff = now - timedelta(hours=24)
            df = df[df[col].apply(lambda d: dt_from_str(d) >= cutoff if pd.notnull(d) and d else False)]
        elif date_filter_mode == 'custom':
            try: from_dt = dt_from_str(date_from_picker.value)
            except: from_dt = None
            try: to_dt = dt_from_str(date_to_picker.value)
            except: to_dt = None
            if from_dt is not None:
                df = df[df[col].apply(lambda d: dt_from_str(d) >= from_dt if pd.notnull(d) and d else False)]
            if to_dt is not None:
                df = df[df[col].apply(lambda d: dt_from_str(d) <= to_dt if pd.notnull(d) and d else False)]
    # Enforce one row per (contact_uuid, ict_uid) by collapsing duplicates to the latest
    try:
        # Only consider rows with both keys populated
        mask_keys = df.apply(
            lambda r: bool(str(r.get('contact_uuid', '')).strip()) and bool(str(r.get('ict_uid', '')).strip()),
            axis=1
        )
        df_keys = df.loc[mask_keys]
        if not df_keys.empty:
            def _safe_dt(val):
                try:
                    d = dt_from_str(val)
                    return d if d is not None else datetime.min
                except Exception:
                    return datetime.min
            # Choose index of the most recently updated row within each pair
            keep_indices = []
            for (cu, iu), grp in df_keys.groupby(['contact_uuid', 'ict_uid']):
                # Prefer 'last_updated_dtg', fallback to 'reported_at', then to index order
                if 'last_updated_dtg' in grp.columns and grp['last_updated_dtg'].notna().any():
                    idx = grp.assign(_k=grp['last_updated_dtg'].apply(_safe_dt)).sort_values('_k', ascending=False).index[0]
                elif 'reported_at' in grp.columns and grp['reported_at'].notna().any():
                    idx = grp.assign(_k=grp['reported_at'].apply(_safe_dt)).sort_values('_k', ascending=False).index[0]
                else:
                    idx = grp.index.max()
                keep_indices.append(idx)
            # Build final df: keep chosen duplicates, plus any rows without complete keys
            df = pd.concat([
                df.loc[~mask_keys],
                df.loc[sorted(set(keep_indices))]
            ]).sort_index()
    except Exception:
        # On any issue, fall back to original df
        pass

    reverse = sort_desc
    col = sort_col
    if col in df.columns:
        df = df.sort_values(col, ascending=not reverse, na_position="last")
    # Preserve original index so markers can map back to exact rows for deletion
    return df

def paged_df():
    df = filtered_sorted_df()
    # If filter results are empty, show raw DataFrame so the table never looks blank unintentionally
    try:
        if df is None or len(df) == 0:
            df = log_df
    except Exception:
        df = log_df
    total = len(df)
    start = page_num * page_size
    end = start + page_size
    return df.iloc[start:end], total

# ---- Table Rendering ----
table_box = widgets.Output()
page_size_dropdown = widgets.Dropdown(
    options=[10, 20, 50, 100], value=10, description='Rows/page:'
)
prev_page_btn = widgets.Button(description='Prev', layout=widgets.Layout(width='70px', height='32px', margin='0 6px 0 10px'))
next_page_btn = widgets.Button(description='Next', layout=widgets.Layout(width='70px', height='32px', margin='0 0 0 6px'))
page_label = widgets.HTML(value='', layout=widgets.Layout(margin='0 0 0 10px'))
def on_page_size_change(change):
    global page_size, page_num
    page_size = int(change['new'])
    page_num = 0
    refresh_all(refresh_map_flag=False)
page_size_dropdown.observe(on_page_size_change, names='value')

def _total_pages():
    total = len(filtered_sorted_df())
    return max(1, (total + page_size - 1) // page_size)

def _update_page_label():
    total = len(filtered_sorted_df())
    total_pages = _total_pages()
    if total == 0:
        page_label.value = '0 rows'
    else:
        page_label.value = f'Page {page_num + 1} of {total_pages} (rows {page_size} per page)'

def _prev_page(btn):
    global page_num
    if page_num > 0:
        page_num -= 1
        refresh_table()

def _next_page(btn):
    global page_num
    tp = _total_pages()
    if page_num < tp - 1:
        page_num += 1
        refresh_table()

prev_page_btn.on_click(_prev_page)
next_page_btn.on_click(_next_page)

# --- Table CSS: Used for HTML and widget mode (via styles in .layout) ---
table_cell_style = dict(
    padding='7px 9px', min_width='55px', max_width='340px',
    font_size='15px', border='1px solid #333', justify_content='center', align_items='center'
)
table_row_style = widgets.Layout(
    display='flex', flex_flow='row', align_items='center', min_height='38px'
)
table_header_style = dict(**table_cell_style, font_weight='bold', background='#252525')

def cell_layout(**kw):
    """Return a Layout merged with table_cell_style and any overrides."""
    return widgets.Layout(**{**table_cell_style, **kw})

table_css = """
<style>
.nato-log-scrollwrap { overflow-x:auto; background:#f9fafb; padding-bottom:8px; border-radius:10px; box-shadow: 0 2px 8px rgba(0,0,0,0.06); }
.nato-log-table { border-collapse:collapse; min-width:1260px; width:100%; background:#ffffff; color:#111827; font-family:-apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif; table-layout: fixed; border-radius:10px; overflow:hidden; box-shadow: 0 2px 8px rgba(0,0,0,0.06); }
.nato-log-table th, .nato-log-table td { border:1px solid #e5e7eb; padding:8px 10px; text-align:left; font-size:15px; vertical-align:middle; }
.nato-log-table th { background:#f3f4f6; font-weight:600; color:#111827; }
.nato-log-table td img { display:block; margin:auto; }
.nato-log-table tr:nth-child(even) { background:#f9fafb; }
.nato-log-table tr:hover { background:#eef2ff; }
/* clamped text block with read-more */
.ict-clamp { position:relative; max-height:6.5em; overflow:hidden; white-space:normal; word-break:break-word; }
.ict-clamp-gradient { position:absolute; bottom:0; left:0; right:0; height:2em; background:linear-gradient(to bottom, rgba(255,255,255,0), rgba(255,255,255,1)); }
.ict-readmore { display:inline-block; margin-top:4px; color:#0ea5e9; cursor:pointer; font-weight:600; }
.ict-readless { display:inline-block; margin-top:4px; color:#0ea5e9; cursor:pointer; font-weight:600; }
/* Light-theme editors inside grid rows */
.nato-input input, .nato-input textarea {
  background:#ffffff !important;
  color:#111827 !important;
  border:1px solid #cbd5e1 !important;
  font-size:14px;
  padding:6px 8px;
  border-radius:6px;
}
.nato-input input::placeholder, .nato-input textarea::placeholder { color:#6b7280; }
.nato-input input:focus, .nato-input textarea:focus { outline: 2px solid #93c5fd; outline-offset: 0; }
.nato-input.nato-editing input, .nato-input.nato-editing textarea { background:#eef2ff !important; }

/* Compact, rounded ipywidgets buttons for table actions */
.widget-button.ict-btn { height:32px; line-height:30px; padding:0 10px; border-radius:6px; border:1px solid #cbd5e1; background:#ffffff; color:#111827; font-weight:600; }
.widget-button.ict-btn:hover { background:#f3f4f6; }
.widget-button.ict-save { background:#10b981; border-color:#059669; color:#ffffff; }
.widget-button.ict-save:hover { background:#059669; }
.widget-button.ict-cancel { background:#e5e7eb; border-color:#cbd5e1; color:#111827; }
.widget-button.ict-cancel:hover { background:#d1d5db; }
.widget-button.ict-danger { background:#ef4444; border-color:#dc2626; color:#ffffff; }
.widget-button.ict-danger:hover { background:#dc2626; }
.widget-button.ict-edit { background:#0ea5e9; border-color:#0284c7; color:#ffffff; }
.widget-button.ict-edit:hover { background:#0284c7; }
</style>
"""

# Helper to render clamped readmore/less HTML for long text
def _clamped_html(safe_text: str) -> str:
    preview = safe_text[:100]
    remainder = safe_text[100:]
    rest_span = ('<span class="ict-rest" style="display:none">' + remainder + '</span>') if remainder else ''
    html = (
        '<div class="ict-clamp">' + preview + rest_span + '<div class="ict-clamp-gradient"></div></div>'
        '<div>'
        '  <span class="ict-readmore" onclick="var c=this.parentElement.previousElementSibling; var rest=c.querySelector(\'.ict-rest\'); if(rest){rest.style.display=\'inline\'}; c.style.maxHeight=\'none\'; c.style.overflow=\'visible\'; var g=c.querySelector(\'.ict-clamp-gradient\'); if(g) g.style.display=\'none\'; this.style.display=\'none\'; var l=this.nextElementSibling; if(l) l.style.display=\'inline\';">Read more</span>'
        '  <span class="ict-readless" style="display:none" onclick="var c=this.parentElement.previousElementSibling; var rest=c.querySelector(\'.ict-rest\'); if(rest){rest.style.display=\'none\'}; c.style.maxHeight=\'6.5em\'; c.style.overflow=\'hidden\'; var g=c.querySelector(\'.ict-clamp-gradient\'); if(g) g.style.display=\'block\'; this.style.display=\'none\'; var p=this.previousElementSibling; if(p) p.style.display=\'inline\';">Read less</span>'
        '</div>'
    )
    return html

def build_html_table():
    df, total = paged_df()
    n = len(df)
    # Full list for list view (scrollable); ordered by sections
    list_columns = [
        # Identity
        'affiliation','unit_identity','report_type','thematic_tags',
        # Contact (entity-level)
        'contact_uuid','contact_ref','contact_short','contact_source_id','contact_status','contact_close_reason','contact_close_dtg',
        # Time & Location
        'when','reported_at','coords','mgrs','mgrs_precision_m','geo_source','location_text',
        # Provenance
        'reporter_id','reporter_type','reporting_unit','collection_method',
        # Evaluation
        'source_reliability','information_credibility','classification','handling_instructions',
        # Relevance
        'pir_id','sir_id','ccir_flag','priority','critical_int',
        # Content & attachments
        'description','attachments_ref',
        # Workflow
        'status','action_taken','dissemination_list','duplicates_of','created_by','reviewed_by','last_updated_dtg','ict_uid','short_uid','source_uid',
        # Legacy/misc
        'source_reference'
    ]
    def _label_for(col_key: str) -> str:
        label_map = {
            'pir_id': 'PIR id',
            'sir_id': 'SIR id',
            'ccir_flag': 'CCIR flag',
            'critical_int': 'Critical Int',
            'mgrs': 'MGRS',
            'contact_uuid': 'Contact uuid',
            'contact_ref': 'Contact Ref',
            'contact_short': 'Contact Short',
            'contact_source_id': 'Contact Source_id',
            'contact_status': 'Contact Status',
            'contact_close_reason': 'Close reason',
            'contact_close_dtg': 'Close DTG',
        }
        if col_key in label_map:
            return label_map[col_key]
        # Default: Title case with spaces
        return col_key.replace('_', ' ').title()
    header = (['Nato Type'] + [_label_for(c) for c in list_columns] + ['History'])
    width_map = {
        'Nato Type': '180px',
        'affiliation': '140px', 'unit_identity': '160px', 'report_type': '140px', 'thematic_tags': '200px',
        'contact_uuid':'220px','contact_ref':'160px','contact_short':'140px','contact_source_id':'160px','contact_status':'120px','contact_close_reason':'200px','contact_close_dtg':'160px',
        'when': '150px', 'reported_at': '150px', 'coords': '180px', 'mgrs':'160px', 'mgrs_precision_m':'120px', 'geo_source':'140px', 'location_text':'240px',
        'reporter_id':'140px','reporter_type':'140px','reporting_unit':'160px','collection_method':'160px',
        'source_reliability':'130px','information_credibility':'130px','classification':'140px','handling_instructions':'260px',
        'pir_id':'120px','sir_id':'120px','priority':'120px','ccir_flag':'100px','critical_manual':'140px','critical_int':'140px',
        'description':'320px','attachments_ref':'200px',
        'status':'120px','action_taken':'220px','dissemination_list':'220px','duplicates_of':'140px','created_by':'140px','reviewed_by':'140px','last_updated_dtg':'160px','ict_uid':'160px','short_uid':'120px','source_uid':'180px',
        'source_reference':'160px'
    }
    def _th(h):
        key = 'Nato Type' if h == 'Nato Type' else h.lower().replace(' ','_')
        w = width_map.get(key, '140px')
        return f'<th style="width:{w}; max-width:{w};">{h}</th>'
    header_html = ''.join(_th(h) for h in header)
    rows_html = ""
    WRAP_COLUMNS = set(['description','handling_instructions','attachments_ref','location_text','dissemination_list','action_taken','analyst_assessment'])
    for i in range(n):
        row = df.iloc[i]
        is_crit_cell = str(row.get('critical_int','')).lower() in ('true','1','yes','on')
        base_svg_cell = SVGs.get(row['symbol'], SVGs[symbol_options[0]])
        icon_url = svg_to_datauri(wrap_with_red_dot(base_svg_cell)) if is_crit_cell else svg_to_datauri(base_svg_cell)
        icon_with_text = (
            f'<div style="display:flex;align-items:center;justify-content:center;gap:8px;">'
            f'<img src="{icon_url}" width="36" height="27"/>'
            f'<span>{str(row.get("symbol",""))}</span>'
            f'</div>'
        )
        tds = [
            f'<td style="text-align:center;vertical-align:middle;padding:7px 9px; width:{width_map.get("Nato Type","180px")}; max-width:{width_map.get("Nato Type","180px")};">{icon_with_text}</td>'
        ]
        for col in list_columns:
            val = row[col]
            # Treat non-scalar values safely (lists, dicts, tuples)
            is_sequence = isinstance(val, (list, tuple, dict, set))
            # Handle coords specially (often a list of [lat, lon])
            if col == "coords":
                if val is None or (is_sequence and len(val) == 0):
                    val = ""
                else:
                    val = str(val)
            else:
                try:
                    # Only use pd.isna for scalar-like values
                    val = "" if (val is None or (not is_sequence and pd.isna(val))) else val
                except Exception:
                    val = "" if val is None else val
            if col in WRAP_COLUMNS:
                safe = _html.escape(str(val))
                content = _clamped_html(safe) if len(safe) > 100 else f'<div style="white-space:normal; word-break:break-word;">{safe}</div>'
                w = width_map.get(col, '220px')
                tds.append(f'<td style="vertical-align:top;padding:7px 9px; width:{w}; max-width:{w};">{content}</td>')
            else:
                w = width_map.get(col, '140px')
                tds.append(f'<td style="vertical-align:middle;padding:7px 9px; white-space:nowrap; overflow:hidden; text-overflow:ellipsis; width:{w}; max-width:{w};">{_html.escape(str(val))}</td>')
        # Append non-interactive history hint cell in HTML mode
        tds.append('<td style="vertical-align:middle;padding:7px 9px; color:#2563eb;">History…</td>')
        rows_html += '<tr>' + ''.join(tds) + '</tr>'
    table_html = f"""{table_css}
    <div class="nato-log-scrollwrap">
    <table class="nato-log-table">
        <thead><tr>{header_html}</tr></thead>
        <tbody>{rows_html}</tbody>
    </table>
    </div>
    """
    return table_html

def build_widget_table():
    from ipywidgets import GridspecLayout
    df, total = paged_df()
    # Duplicate guard: detect multiple physical rows sharing the same (contact_uuid, ict_uid)
    try:
        full_df = filtered_sorted_df()
    except Exception:
        full_df = df
    # Consider only rows where both keys are present (non-empty)
    dup_alert_html = ''
    dup_alert_widget = None
    try:
        key_mask = full_df.apply(
            lambda r: bool(str(r.get('contact_uuid', '')).strip()) and bool(str(r.get('ict_uid', '')).strip()),
            axis=1
        )
        key_df = full_df.loc[key_mask, ['contact_uuid', 'ict_uid']].copy()
        if not key_df.empty:
            dups = key_df[key_df.duplicated(keep=False)]
            if not dups.empty:
                counts = dups.groupby(['contact_uuid', 'ict_uid']).size().reset_index(name='count')
                items_html = ''.join(
                    [
                        f"<li><code>{_html.escape(str(row['contact_uuid']))}</code> / "
                        f"<code>{_html.escape(str(row['ict_uid']))}</code> — {int(row['count'])} rows</li>"
                        for _, row in counts.iterrows()
                    ]
                )
                dup_alert_html = (
                    "<div style=\"background:#fef2f2;border:1px solid #fecaca;"
                    "color:#7f1d1d;padding:10px 12px;border-radius:8px;"
                    "font-family:-apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;\">"
                    "<strong>Duplicate Contact/Serial pairs detected.</strong> "
                    "The application expects exactly one row per (<code>contact_uuid</code>, <code>ict_uid</code>). "
                    "Please resolve these duplicates at source; imports with duplicates should be rejected."
                    f"<ul style=\"margin:8px 0 0 18px;\">{items_html}</ul>"
                    "</div>"
                )
    except Exception:
        # Do not block table rendering on alert computation
        dup_alert_html = ''
    # Build alert widget if needed
    if dup_alert_html:
        try:
            dup_alert_widget = widgets.HTML(value=dup_alert_html, layout=widgets.Layout(margin='0 0 10px 0'))
        except Exception:
            dup_alert_widget = None
    n = len(df)
    def _label_for(col_key: str) -> str:
        label_map = {
            'pir_id': 'PIR id',
            'sir_id': 'SIR id',
            'ccir_flag': 'CCIR flag',
            'critical_int': 'Critical Int',
            'mgrs': 'MGRS',
        }
        return label_map.get(col_key, col_key.replace('_', ' ').title())
    col_names = ['Nato Type'] + [_label_for(col) for col in log_columns[1:]] + ['History', 'Delete', 'Edit']
    ncols = len(col_names)
    # Same column width order as HTML CSS (min/max widths from your .nato-log-table CSS)
    # Generate widths to match the number of columns exactly
    base_widths_map = {
        'symbol':'180px','when':'150px','coords':'180px','reported_at':'150px','reporter_id':'140px','reporter_type':'140px','reporting_unit':'160px','collection_method':'160px',
        'source_reliability':'130px','information_credibility':'130px','classification':'140px','handling_instructions':'260px','pir_id':'120px','sir_id':'120px','priority':'120px','ccir_flag':'100px','critical_manual':'140px','critical_int':'140px',
        'description':'320px','attachments_ref':'200px','status':'120px','action_taken':'220px','dissemination_list':'220px','duplicates_of':'140px','created_by':'140px','reviewed_by':'140px','last_updated_dtg':'160px',
        'contact_uuid':'220px','contact_ref':'160px','contact_short':'140px','contact_source_id':'160px','contact_status':'120px','contact_close_reason':'200px','contact_close_dtg':'160px',
        'source_reference':'160px'
    }
    dynamic_widths = ['180px'] + [base_widths_map.get(col, '140px') for col in log_columns[1:]] + ['90px','82px','110px']
    col_widths = dynamic_widths
    header_style = (
        "background:#f3f4f6;"
        "font-weight:600;"
        "color:#111827;"
        "font-size:15px;"
        "text-align:center;"
        "vertical-align:middle;"
        "border:1px solid #e5e7eb;"
        "padding:8px 10px;"
        "font-family:-apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;"
        "min-height:36px; max-height:36px;"
    )
    cell_style = (
        "font-size:15px;"
        "text-align:left;"
        "vertical-align:middle;"
        "border:1px solid #e5e7eb;"
        "padding:8px 10px;"
        "font-family:-apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;"
        "min-height:36px; max-height:36px;"
        "background:#ffffff;"
        "color:#111827;"
    )
    # Build the grid
    grid = GridspecLayout(n + 1, ncols, width='max-content', min_width='1260px')
    # Reduce row height and add spacing for a cleaner, non-overlapping look

    # Set column widths exactly as HTML
    col_template = " ".join(col_widths[:ncols])
    grid.layout.grid_template_columns = col_template

    # Fill header
    for j, h in enumerate(col_names):
        grid[0, j] = widgets.HTML(
            f'<div style="{header_style}; text-align:center; display:flex; align-items:center; justify-content:center; width:100%; height:100%">{h}</div>',
            layout=widgets.Layout(
                width=col_widths[j] if j < len(col_widths) else "110px",
                min_width=col_widths[j] if j < len(col_widths) else "110px",
                max_width=col_widths[j] if j < len(col_widths) else "110px",
                min_height="36px", max_height="36px"
            )
        )

    # Fill data
    for i in range(n):
        row = df.iloc[i]
        orig_idx = df.index[i]
        is_editing = (editing_row_idx[0] == i)

        # Nato Type cell (icon + text)
        icon_url = svg_to_datauri(SVGs.get(row['symbol'], SVGs[symbol_options[0]]))
        grid[i+1, 0] = widgets.HTML(
            f'<div style="display:flex;align-items:center;justify-content:center;gap:8px;"><img src="{icon_url}" width="36" height="27"/><span>{str(row.get("symbol",""))}</span></div>',
            layout=widgets.Layout(width=col_widths[0], min_width=col_widths[0], max_width=col_widths[0], min_height="36px", max_height="36px", justify_content="center", align_items="center")
        )

        # Editors for editing mode
        editors = {}
        def _cell_layout(j):
            return widgets.Layout(width=col_widths[j], min_width=col_widths[j], max_width=col_widths[j], min_height="36px", max_height="36px", margin="0")

        # Show all columns in the grid in the same order as schema (excluding the symbol shown in column 0)
        columns_in_order = list(log_columns[1:])
        for j, col_name in enumerate(columns_in_order, start=1):
            if is_editing and col_name != 'coords':
                # Build appropriate editor
                if col_name in ('description','attachments_ref','location_text','dissemination_list','action_taken'):
                    # Auto-growing textarea for long free-text fields
                    w = widgets.Textarea(value=str(row[col_name] or ''), rows=3, layout=_cell_layout(j+0))
                    def _auto_grow(change, wt=w):
                        try:
                            lines = (wt.value or '').count('\n') + 1
                            wt.rows = min(10, max(3, lines))
                        except Exception:
                            pass
                    w.observe(_auto_grow, names='value')
                    try: w.add_class('nato-input')
                    except Exception: pass
                elif col_name in ('when','reported_at','reporter_id','reporter_type','reporting_unit','collection_method','source_reliability','information_credibility','classification','handling_instructions','priority','ccir_flag','critical_int','attachments_ref','status','action_taken','dissemination_list','duplicates_of','created_by','reviewed_by','last_updated_dtg','ict_uid','short_uid','source_uid','source_reference','mgrs','mgrs_precision_m','geo_source','location_text'):
                    placeholder = 'YYYY-MM-DD:HH:MM:SS' if col_name in ('when','reported_at') else ''
                    w = widgets.Text(value=str(row[col_name] or ''), placeholder=placeholder, layout=_cell_layout(j+0))
                    try: w.add_class('nato-input')
                    except Exception: pass
                else:
                    w = widgets.Text(value=str(row[col_name] or ''), layout=_cell_layout(j+0))
                    try: w.add_class('nato-input')
                    except Exception: pass
                try: w.add_class('nato-editing')
                except Exception: pass
                editors[col_name] = w
                # Wrap editor in a div with the dark-theme class so CSS applies
                grid[i+1, j] = widgets.Box([w], layout=_cell_layout(j))
            else:
                # Read-only display (including coords). Add special badge styling for Critical Int.
                val = row[col_name]
                val = '' if val is None else str(val)
                row_bg = ('#eef2ff' if is_editing else ('#f9fafb' if (i % 2 == 1) else '#ffffff'))
                if col_name == 'critical_int':
                    is_crit = str(val).lower() in ('true','1','yes','on')
                    style = 'background:#dc2626;color:#ffffff;font-weight:700;' if is_crit else 'border:1px solid #9ca3af;color:#111827;background:#ffffff;'
                    text = '🔴 Critical' if is_crit else '⚪ Not Critical'
                    badge = f'<div style="display:inline-flex;align-items:center;gap:6px;padding:4px 10px;border-radius:14px;{style}">{text}</div>'
                    grid[i+1, j] = widgets.HTML(f'<div style="{cell_style} background:{row_bg}; text-align:center;">{badge}</div>', layout=_cell_layout(j))
                    continue
                if col_name in ('description','handling_instructions','attachments_ref','location_text','dissemination_list','action_taken','analyst_assessment'):
                    html_val = f'<div style="{cell_style} background:{row_bg}; white-space:normal; word-break:break-word;">{val}</div>'
                else:
                    html_val = f'<div style="{cell_style} background:{row_bg}; white-space:nowrap; overflow:hidden; text-overflow:ellipsis;">{val}</div>'
                grid[i+1, j] = widgets.HTML(html_val, layout=_cell_layout(j))

        # Action buttons
        # History button
        hist_btn = widgets.Button(description="History…", button_style='', layout=widgets.Layout(width=col_widths[-3], min_width=col_widths[-3], max_width=col_widths[-3], height="32px", margin="2px 0 0 0"))
        hist_btn.add_class('ict-btn')
        def _open_hist(btn, idx=orig_idx):
            try:
                entry_id = str(log_df.loc[idx].get('entry_id','')) if idx in log_df.index else ''
            except Exception:
                entry_id = ''
            open_history_dialog(entry_id)
        hist_btn.on_click(_open_hist)

        del_btn = widgets.Button(description="X", button_style='', layout=widgets.Layout(width=col_widths[-2], min_width=col_widths[-2], max_width=col_widths[-2], height="32px", margin="2px 0 0 0"))
        del_btn.add_class('ict-btn')
        del_btn.add_class('ict-danger')
        
        # Local helper: delete row without rebinding the DataFrame (avoid globals)
        def _delete_row_inplace(row_idx: int):
            try:
                log_df.drop(index=[row_idx], inplace=True)
                log_df.reset_index(drop=True, inplace=True)
            except Exception:
                return
            refresh_all(refresh_map_flag=True)

        def _ask_delete(btn, idx=orig_idx):
            try:
                open_delete_confirm(idx)
            except Exception:
                # Fallback: delete directly if dialog not available for any reason
                _delete_row_inplace(idx)

        del_btn.on_click(_ask_delete)
        # Place History, Delete, Edit in the last three columns
        grid[i+1, ncols-3] = hist_btn
        grid[i+1, ncols-2] = del_btn

        if is_editing:
            save_btn = widgets.Button(description="Save", button_style='', layout=widgets.Layout(width='80px', height="32px", margin="2px"))
            save_btn.add_class('ict-btn')
            save_btn.add_class('ict-save')
            cancel_btn = widgets.Button(description="Cancel", button_style='', layout=widgets.Layout(width='80px', height="32px", margin="2px"))
            cancel_btn.add_class('ict-btn')
            cancel_btn.add_class('ict-cancel')
            def _save(btn, idx=orig_idx, editors=editors):
                global log_df
                for k, w in editors.items():
                    log_df.loc[idx, k] = w.value
                # Leave coords unchanged
                editing_row_idx[0] = None
                refresh_all(refresh_map_flag=True)
            def _cancel(btn):
                editing_row_idx[0] = None
                refresh_table()
            save_btn.on_click(_save)
            cancel_btn.on_click(_cancel)
            grid[i+1, ncols-1] = widgets.HBox([save_btn, cancel_btn])
        else:
            edit_btn = widgets.Button(description="Edit", button_style='', layout=widgets.Layout(width=col_widths[-1], min_width=col_widths[-1], max_width=col_widths[-1], height="32px", margin="2px 0 0 0"))
            edit_btn.add_class('ict-btn')
            edit_btn.add_class('ict-edit')
        def begin_edit(btn, i=i):
            editing_row_idx[0] = i
            refresh_table()
        # Attach handler only when not editing
        if not is_editing:
            edit_btn.on_click(begin_edit)
            grid[i+1, ncols-1] = edit_btn
    # Wrap in scrollable box with same light theme background
    scroll = widgets.Box([grid], layout=widgets.Layout(overflow_x='auto', overflow_y='visible', min_width='1260px', width='100%', max_width='100%', background='#f9fafb', border_radius='8px', padding='0 0 8px 0'))
    children = [scroll]
    if dup_alert_widget is not None:
        children.insert(0, dup_alert_widget)
    return widgets.VBox(children)



def build_vuetify_table():
    # Build a Vuetify DataTable with inline editing
    df = filtered_sorted_df().copy()
    df = df.reset_index().rename(columns={"index": "_row_id"})

    headers = [{"text": "Nato Type", "value": "symbol_icon", "sortable": False, "width": 80, "align": "center"}]
    for col in log_columns[1:]:
        headers.append({"text": col.replace('_', ' ').capitalize(), "value": col})
    headers += [
        {"text": "History", "value": "_history", "sortable": False, "width": 90},
        {"text": "Delete", "value": "_delete", "sortable": False, "width": 70},
        {"text": "Edit", "value": "_edit", "sortable": False, "width": 120},
    ]

    items = []
    def _norm_cell(val):
        # Keep coordinate arrays/lists as-is; only blank-out true NaNs/None
        if isinstance(val, (list, tuple)):
            return list(val)
        try:
            return "" if (val is None or pd.isna(val)) else val
        except Exception:
            return val if val is not None else ""
    for _, row in df.iterrows():
        item = {c: _norm_cell(row[c]) for c in log_columns}
        item["_row_id"] = int(row["_row_id"])
        item["symbol_icon"] = svg_to_datauri(SVGs.get(row['symbol'], SVGs[symbol_options[0]]))
        items.append(item)

    dt = v.DataTable(
        headers=headers,
        items=items,
        dense=True,
        fixed_header=True,
        height='60vh',
        class_='elevation-1',
        items_per_page=page_size,
    )

    def symbol_cell(item):
        # Icon + name inline
        name = str(item.get('symbol',''))
        html = f"<div style='display:flex;align-items:center;justify-content:center;gap:8px;'><img src='{item['symbol_icon']}' style='width:36px;height:27px;'/><span>{name}</span></div>"
        return v.Html(tag='div', children=[html])

    def history_cell(item):
        def _click(widget, event, data, _row_id=item['_row_id']):
            entry_id = ''
            try:
                entry_id = str(log_df.loc[_row_id].get('entry_id','')) if _row_id in log_df.index else ''
            except Exception:
                entry_id = ''
            open_history_dialog(entry_id)
        btn = v.Btn(children=['History…'], small=True, outlined=True)
        btn.on_event('click', _click)
        return btn

    def delete_cell(item):
        def _click(widget, event, data, _row_id=item['_row_id']):
            global log_df
            if _row_id in log_df.index:
                log_df = log_df.drop(_row_id).reset_index(drop=True)
                refresh_table()
                refresh_map()
        btn = v.Btn(children=['X'], small=True, color='error', outlined=True)
        btn.on_event('click', _click)
        return btn

    def edit_cell(item):
        _row_id = item['_row_id']
        is_editing = (editing_row_id[0] == _row_id)
        # Only show edit controls when global edit toggle is enabled
        if not edit_mode.value:
            return v.Html(tag='div')
        if not is_editing:
            btn = v.Btn(children=['✎'], small=True, color='info', outlined=True)
            def _start(widget, event, data):
                editing_row_id[0] = _row_id
                refresh_table()
            btn.on_event('click', _start)
            return btn
        save_btn = v.Btn(children=['Save'], small=True, color='success', class_='mr-2')
        cancel_btn = v.Btn(children=['Cancel'], small=True, outlined=True)
        def _save(widget, event, data):
            global log_df
            for it in dt.items:
                if it.get('_row_id') == _row_id:
                    for col in log_columns:
                        if col == 'coords':
                            continue
                        log_df.loc[_row_id, col] = it.get(col, '')
                    break
            editing_row_id[0] = -1
            refresh_table()
            refresh_map()
        def _cancel(widget, event, data):
            editing_row_id[0] = -1
            refresh_table()
        save_btn.on_event('click', _save)
        cancel_btn.on_event('click', _cancel)
        return v.Row(children=[save_btn, cancel_btn], no_gutters=True)

    def make_cell_slot(col):
        def _slot(item, col=col):
            _row_id = item['_row_id']
            # read-only if not in row edit OR edit mode is globally off
            if (editing_row_id[0] != _row_id) or (not edit_mode.value) or col == 'coords':
                # Render Critical Int as badge in vuetify table too
                if col == 'critical_int':
                    is_crit = str(item.get(col, '')).lower() in ('true','1','yes','on')
                    style = 'background:#dc2626;color:#ffffff;font-weight:700;' if is_crit else 'border:1px solid #9ca3af;color:#111827;background:#ffffff;'
                    text = '🔴 Critical' if is_crit else '⚪ Not Critical'
                    return v.Html(tag='div', children=[f"<div style='display:inline-flex;align-items:center;gap:6px;padding:4px 10px;border-radius:14px;{style}'>{text}</div>"])
                return v.Html(tag='div', children=[str(item.get(col, ''))], style_='white-space: nowrap;')
            tf = v.TextField(v_model=item.get(col, ''), dense=True, hide_details=True, class_='nato-input')
            def _on_input(widget, event, data, _row_id=_row_id, col=col):
                for it in dt.items:
                    if it.get('_row_id') == _row_id:
                        it[col] = data
                        break
            tf.on_event('input', _on_input)
            return tf
        return _slot

    # Build Vuetify slots using v_slots API
    v_slots = [
        {'name': 'item.symbol_icon', 'variable': 'item', 'children': symbol_cell},
        {'name': 'item._history', 'variable': 'item', 'children': history_cell},
        {'name': 'item._delete', 'variable': 'item', 'children': delete_cell},
        {'name': 'item._edit', 'variable': 'item', 'children': edit_cell},
    ]
    for col in log_columns[1:]:
        v_slots.append({'name': f'item.{col}', 'variable': 'item', 'children': make_cell_slot(col)})
    dt.v_slots = v_slots

    def _on_page_size(change):
        if change.get('name') == 'value':
            dt.items_per_page = change['new']
    page_size_dropdown.observe(_on_page_size, names='value')

    # alert_children already computed above
    return v.Container(children=[dt], style_='max-width: 100%;')

def build_table():
    # Always use the ipywidgets grid so row actions (History) are clickable in both modes
    return build_widget_table()

edit_mode.observe(lambda c: refresh_table(), names='value')

# ---- Map logic ----
m = Map(center=(51.5, -0.12), zoom=12, scroll_wheel_zoom=True, layout=widgets.Layout(width="100%", height="75vh"))
try:
    m.double_click_zoom = False
except Exception:
    pass

# --- Basemap layers (streets/satellite/topo) ---
base_layers = {
    "Streets": basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    "Satellite": basemap_to_tiles(basemaps.Esri.WorldImagery),
    "Topo": basemap_to_tiles(basemaps.OpenTopoMap),  # contours and terrain
}
hillshade_overlay = basemap_to_tiles(basemaps.Esri.WorldShadedRelief)
hillshade_overlay.opacity = 0.35
current_base_layer = [base_layers["Streets"]]
# Replace default base layer with our chosen one
try:
    m.layers = tuple([current_base_layer[0]])
except Exception:
    m.add_layer(current_base_layer[0])

# Optional hillshade overlay (toggle via LayerControl)
m.add_layer(hillshade_overlay)
hillshade_overlay.visible = False

# Controls: add Leaflet fullscreen box (top-left) and keep LayersControl (top-right)
m.add_control(FullScreenControl(position='topleft'))
m.add_control(LayersControl(position='topright'))
marker_objects = []
# Store transient location accuracy circles
location_circles = []

def _clear_location_circles():
    for c in list(location_circles):
        try:
            m.remove_layer(c)
        except Exception:
            pass
    location_circles.clear()

def _meters_per_pixel(lat: float, zoom: float) -> float:
    # Approximate meters per pixel at latitude using Web Mercator formula
    try:
        return 156543.03392 * math.cos(math.radians(lat)) / (2 ** float(zoom))
    except Exception:
        return 156543.03392 / (2 ** float(zoom))

def enhance_location_overlays():
    """Draw semi-transparent circles sized by mgrs_precision_m scaled by current zoom."""
    _clear_location_circles()
    try:
        df = filtered_sorted_df()
    except Exception:
        df = log_df
    # If filters currently hide everything (map uses a fallback), mirror that behavior here
    try:
        if df is None or len(df) == 0:
            df = log_df
    except Exception:
        df = log_df
    # Use map center latitude for scale approximation per zoom
    try:
        center_lat = float(m.center[0]) if hasattr(m, 'center') else 0.0
    except Exception:
        center_lat = 0.0
    mpp = _meters_per_pixel(center_lat, m.zoom)
    for idx, row in df.iterrows():
        # Mirror map visibility: hide closed contacts/serials
        try:
            if str(row.get('status','')).strip().lower() == 'closed' or str(row.get('contact_status','')).strip().lower() == 'closed':
                continue
        except Exception:
            pass
        coords = row.get('coords')
        if coords is None or not isinstance(coords, (list, tuple)) or len(coords) != 2:
            continue
        try:
            raw = row.get('mgrs_precision_m')
            if raw in (None, ''):
                raw = row.get('mgrs_precision', None)
            if raw in (None, ''):
                raw = row.get('precision_m', None)
            precision_m = float(raw) if raw not in (None, '') else None
        except Exception:
            precision_m = None
        if precision_m is None or precision_m <= 0:
            continue
        # Scale: radius in pixels roughly = precision_m / meters_per_pixel
        try:
            radius_px = max(4, precision_m / max(0.1, mpp))
        except Exception:
            radius_px = 6
        # Style: red border if critical, else blue
        is_crit = str(row.get('critical_int','')).lower() in ('true','1','yes','on')
        border_color = "#dc2626" if is_crit else "#2563eb"
        border_weight = 3 if is_crit else 1
        circle = CircleMarker(
            location=(float(coords[0]), float(coords[1])),
            radius=int(radius_px),
            color=border_color,
            fill_color="#2563eb",
            fill_opacity=0.2,
            opacity=0.6,
            weight=border_weight,
        )
        try:
            m.add_layer(circle)
            location_circles.append(circle)
        except Exception:
            pass
def refresh_map():
    for mk, idx in marker_objects:
        try: m.remove_layer(mk)
        except: pass
    marker_objects.clear()
    size = icon_size_for_zoom(m.zoom)
    df = filtered_sorted_df()
    # If nothing survives filtering, fall back to raw df to avoid an empty map in error states
    if df is None or len(df) == 0:
        df = log_df
    # Only one marker per Contact: collapse to most recent serial per contact_uuid
    try:
        if 'contact_uuid' in df.columns and len(df) > 0:
            def _coalesce_dt(r):
                # Prefer last_updated_dtg, then reported_at, then when
                for key in ('last_updated_dtg','reported_at','when'):
                    try:
                        v = r.get(key)
                        if v:
                            return dt_from_str(v)
                    except Exception:
                        pass
                return datetime.min
            keep_indices = []
            for cu, grp in df.groupby('contact_uuid'):
                try:
                    idx_keep = grp.assign(_k=grp.apply(_coalesce_dt, axis=1))\
                                   .sort_values('_k', ascending=False).index[0]
                except Exception:
                    idx_keep = grp.index.max()
                keep_indices.append(idx_keep)
            df = df.loc[sorted(set(keep_indices))]
    except Exception:
        pass

    for idx, row in df.iterrows():
        # Hide contacts marked Closed from the map only
        try:
            if str(row.get('status','')).strip().lower() == 'closed' or str(row.get('contact_status','')).strip().lower() == 'closed':
                continue
        except Exception:
            pass
        base_svg = SVGs.get(row["symbol"], SVGs[symbol_options[0]])
        is_crit = str(row.get('critical_int','')).lower() in ('true','1','yes','on')
        icon_svg = base_svg.replace('</svg>', "<circle cx='12' cy='12' r='12' fill='#dc2626'/></svg>") if is_crit else base_svg
        icon = Icon(icon_url=svg_to_datauri(icon_svg), icon_size=size, icon_anchor=[size[0]//2, size[1]//2])
        try:
            coords = row["coords"]
            if not coords or not isinstance(coords, (list, tuple)) or len(coords) != 2:
                continue
            marker = Marker(location=coords, icon=icon)
        except Exception:
            continue
        # Allow users to drag markers to adjust location
        try:
            marker.draggable = True
        except Exception:
            pass
        # Attach the original row index and contact id to the marker for precise deletion
        marker.row_index = idx
        try:
            marker.contact_uuid = str(row.get('contact_uuid',''))
        except Exception:
            marker.contact_uuid = ''
        # Attach click handler: open edit modal (Remove OFF); double-click to remove (Remove ON)
        def make_marker_click_handler(row_idx):
            last_click_time = {"t": 0.0}
            def _handler(**_):
                try:
                    remove = bool(remove_switch.v_model)
                except NameError:
                    remove = False
                now_ts = time.time()
                if remove:
                    # in remove mode require double-click
                    if now_ts - last_click_time["t"] < 0.4:
                        # double click → remove exact row
                        global log_df
                        try:
                            cu = str(log_df.loc[row_idx].get('contact_uuid','')) if row_idx in log_df.index else ''
                        except Exception:
                            cu = ''
                        if cu:
                            # Delete all rows for this contact
                            log_df = log_df[log_df['contact_uuid'] != cu].reset_index(drop=True)
                        elif row_idx in log_df.index:
                            log_df = log_df.drop(row_idx).reset_index(drop=True)
                            refresh_all(refresh_map_flag=True)
                    last_click_time["t"] = now_ts
                else:
                    # Open full edit dialog on marker click
                    try:
                        row = log_df.loc[row_idx]
                        prefill = {
                            'symbol': row.get('symbol',''),
                            'when': row.get('when',''),
                            'coords': row.get('coords', []),
                            'reported_at': row.get('reported_at',''),
                            'reporter_id': row.get('reporter_id',''),
                            'reporter_type': row.get('reporter_type',''),
                            'source_reliability': row.get('source_reliability',''),
                            'information_credibility': row.get('information_credibility',''),
                            'description': row.get('description',''),
                            'source_reference': row.get('source_reference',''),
                            # Identity
                            'affiliation': row.get('affiliation',''),
                            'unit_identity': row.get('unit_identity',''),
                            'report_type': row.get('report_type',''),
                            'thematic_tags': row.get('thematic_tags',''),
                            # Location
                            'mgrs': row.get('mgrs',''),
                            'mgrs_precision_m': row.get('mgrs_precision_m',''),
                            'geo_source': row.get('geo_source',''),
                            'location_text': row.get('location_text',''),
                            # Content
                            'size': row.get('size',''),
                            'activity': row.get('activity',''),
                            'equipment': row.get('equipment',''),
                            'time_observed': row.get('time_observed',''),
                            'attachments_ref': row.get('attachments_ref',''),
                            # Evaluation
                            'classification': row.get('classification',''),
                            'handling_instructions': row.get('handling_instructions',''),
                            'analyst_assessment': row.get('analyst_assessment',''),
                            'analyst_confidence': row.get('analyst_confidence',''),
                            # Relevance / Tasking
                            'pir_id': row.get('pir_id',''),
                            'sir_id': row.get('sir_id',''),
                            'ccir_flag': row.get('ccir_flag',''),
                            'priority': row.get('priority',''),
                            'critical_manual': row.get('critical_manual',''),
                            'critical_int': row.get('critical_int',''),
                            # Provenance
                            'reporting_unit': row.get('reporting_unit',''),
                            'collection_method': row.get('collection_method',''),
                            # Workflow
                            'status': row.get('status',''),
                            'action_taken': row.get('action_taken',''),
                             'dissemination_list': row.get('dissemination_list',''),
                             # UIDs
                             'ict_uid': row.get('ict_uid',''),
                             'short_uid': row.get('short_uid',''),
                             'source_uid': row.get('source_uid',''),
                        }
                        _open_entry_dialog(prefill, row_idx)
                    except Exception as e:
                        print('Open dialog error:', e)
            return _handler
        marker.on_click(make_marker_click_handler(idx))

        # Attach drag handler to update coordinates and open Location section for review
        def _attach_drag_behavior(mk: Marker, row_idx: int):
            state = {"last_ts": 0.0}
            def _on_move(change, _row_idx=row_idx, _state=state):
                if change.get('name') != 'location':
                    return
                try:
                    new_loc = change.get('new') or mk.location
                    # Update coordinates immediately in the DataFrame
                    log_df.at[_row_idx, 'coords'] = list(new_loc)
                    try:
                        log_df.at[_row_idx, 'lat'] = float(new_loc[0])
                        log_df.at[_row_idx, 'lon'] = float(new_loc[1])
                    except Exception:
                        pass
                except Exception:
                    pass
                # Debounce dialog opening to when drag stops (~400ms idle)
                _state['last_ts'] = time.time()
                ts = _state['last_ts']
                def _open_if_idle():
                    try:
                        time.sleep(0.4)
                        if abs(_state['last_ts'] - ts) > 1e-6:
                            return
                        # Prefill with latest row values and open dialog at Location section
                        try:
                            row2 = log_df.loc[_row_idx]
                            prefill2 = {
                                'symbol': row2.get('symbol',''),
                                'when': row2.get('when',''),
                                'coords': row2.get('coords', []),
                                'reported_at': row2.get('reported_at',''),
                                'reporter_id': row2.get('reporter_id',''),
                                'reporter_type': row2.get('reporter_type',''),
                                'source_reliability': row2.get('source_reliability',''),
                                'information_credibility': row2.get('information_credibility',''),
                                'description': row2.get('description',''),
                                'source_reference': row2.get('source_reference',''),
                                # Identity
                                'affiliation': row2.get('affiliation',''),
                                'unit_identity': row2.get('unit_identity',''),
                                'report_type': row2.get('report_type',''),
                                'thematic_tags': row2.get('thematic_tags',''),
                                # Location
                                'mgrs': row2.get('mgrs',''),
                                'mgrs_precision_m': row2.get('mgrs_precision_m',''),
                                'geo_source': row2.get('geo_source',''),
                                'location_text': row2.get('location_text',''),
                                # Evaluation
                                'classification': row2.get('classification',''),
                                'handling_instructions': row2.get('handling_instructions',''),
                                'analyst_assessment': row2.get('analyst_assessment',''),
                                'analyst_confidence': row2.get('analyst_confidence',''),
                                # Relevance / Tasking
                                'pir_id': row2.get('pir_id',''),
                                'sir_id': row2.get('sir_id',''),
                                'ccir_flag': row2.get('ccir_flag',''),
                                'priority': row2.get('priority',''),
                                'critical_manual': row2.get('critical_manual',''),
                                'critical_int': row2.get('critical_int',''),
                                # Provenance
                                'reporting_unit': row2.get('reporting_unit',''),
                                'collection_method': row2.get('collection_method',''),
                                # Workflow
                                'status': row2.get('status',''),
                                'action_taken': row2.get('action_taken',''),
                                'dissemination_list': row2.get('dissemination_list',''),
                                # UIDs
                                'ict_uid': row2.get('ict_uid',''),
                                'short_uid': row2.get('short_uid',''),
                                'source_uid': row2.get('source_uid',''),
                            }
                            _open_entry_dialog(prefill2, _row_idx)
                            try:
                                # Open Location panel
                                entry_panels.v_model = 2
                            except Exception:
                                pass
                        except Exception:
                            pass
                    except Exception:
                        pass
                import threading
                threading.Thread(target=_open_if_idle, daemon=True).start()
            try:
                mk.observe(_on_move, names=['location'])
            except Exception:
                pass
        _attach_drag_behavior(marker, idx)
        m.add_layer(marker)
        marker_objects.append((marker, idx))
    # If overlays are toggled on, regenerate them for the new zoom
    try:
        if enhance_location_switch.v_model:
            enhance_location_overlays()
    except Exception:
        pass

# ---- Manual Map Update Button ----
update_map_btn = widgets.Button(
    description="Update Map", button_style='primary', icon='refresh',
    layout=widgets.Layout(width="130px", min_height="38px", max_height="38px", margin='0 0 0 14px')
)
def update_map_btn_clicked(btn):
    refresh_map()
update_map_btn.on_click(update_map_btn_clicked)

# ---- Create Button ----
create_btn = widgets.Button(
    description="Create", button_style='success', icon='plus',
    layout=widgets.Layout(width="110px", min_height="38px", max_height="38px", margin='0 0 0 16px')
)
def create_btn_clicked(btn):
    # Insert a new blank row at the end with current timestamps so sorting (reported_at desc)
    # brings it to the top; then open that first row in edit mode in the widget table.
    global log_df, page_num, editing_row_idx
    new_row = {
        # Identity
        'symbol': symbol_options[0],
        'entry_id': str(uuid.uuid4()),
        'affiliation': '',
        'unit_identity': '',
        'report_type': '',
        'thematic_tags': '',
        # Time
        'when': now_str(),
        'reported_at': now_str(),
        'staleness_minutes': '',
        # Location
        'coords': '',
        'lat': '',
        'lon': '',
        'mgrs': '',
        'mgrs_precision_m': '',
        'geo_source': '',
        'location_text': '',
        # Contact
        'contact_uuid': str(uuid.uuid4()),
        'contact_ref': '',
        'contact_short': '',
        'contact_source_id': '',
        'contact_status': 'Active',
        'contact_close_reason': '',
        'contact_close_dtg': '',
        # Content
        'size': '',
        'activity': '',
        'equipment': '',
        'time_observed': '',
        'description': '',
        'attachments_ref': '',
        # Evaluation
        'source_reliability': '',
        'information_credibility': '',
        'classification': '',
        'handling_instructions': '',
        'analyst_assessment': '',
        'analyst_confidence': '',
        # Relevance / Tasking
        'pir_id': '',
        'sir_id': '',
        'ccir_flag': '',
        'priority': '',
        'critical_manual': '',
        'critical_int': '',
        # Provenance
        'reporting_unit': '',
        'reporter_type': '',
        'reporter_id': '',
        'collection_method': '',
        # Workflow / QA
        'status': '',
        'action_taken': '',
        'dissemination_list': '',
        'duplicates_of': '',
        'created_by': '',
        'reviewed_by': '',
        'last_updated_dtg': now_str(),
        # Legacy / misc
        'source_reference': ''
    }
    try:
        log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)
        # initial version
        _append_version(new_row['entry_id'], op='create', payload=_canonical_payload(pd.Series(new_row)))
    except Exception:
        pass
    # Align UX: jump to first page, set first row editing, and apply the same visual highlight
    page_num = 0
    editing_row_idx[0] = 0
    # Ensure edit mode is on so widget table renders editors
    try:
        edit_mode.value = True
    except Exception:
        pass
    refresh_all(refresh_map_flag=False)
create_btn.on_click(create_btn_clicked)

# ---- CSV Download ----
download_csv_btn = widgets.Button(
    description="Download CSV", icon='download',
    layout=widgets.Layout(width="140px", min_height="38px", max_height="38px", margin='0 0 0 14px')
)
csv_download_link = widgets.HTML(value="", layout=widgets.Layout(margin='0 0 0 10px'))
def _on_download_csv(btn):
    try:
        df = filtered_sorted_df()
        csv_text = df.to_csv(index=False)
        import base64
        b64 = base64.b64encode(csv_text.encode('utf-8')).decode('ascii')
        href = f'<a download="ict_irt_log.csv" href="data:text/csv;base64,{b64}">Click to download</a>'
        csv_download_link.value = href
    except Exception as e:
        csv_download_link.value = f"<span style='color:#f55'>Download failed: {e}</span>"
download_csv_btn.on_click(_on_download_csv)

# ---- Main refresh ----
def refresh_all(refresh_map_flag=True):
    filter_row.children = [build_filter_row()]
    refresh_table()
    if refresh_map_flag:
        refresh_map()

def refresh_table():
    table_box.clear_output(wait=True)
    with table_box:
        display(build_table())
    _update_page_label()

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        global log_df
        latlng = kwargs['coordinates']
        chosen_symbol = current_symbol_v[0]

        # Double-click detection (works even if 'dblclick' event not provided)
        if not hasattr(handle_click, "_last_click"):
            handle_click._last_click = {"t": 0.0, "lat": None, "lng": None}
        last = handle_click._last_click
        now = time.time()
        is_double = (now - last["t"] < 0.4) and (last["lat"] is not None) and ((float(last["lat"]) - float(latlng[0]))**2 + (float(last["lng"]) - float(latlng[1]))**2 < 1e-6)
        handle_click._last_click = {"t": now, "lat": float(latlng[0]), "lng": float(latlng[1])}

        # Toggle ON: removal only (no adding on single click)
        try:
            remove_mode = bool(remove_switch.v_model)
        except NameError:
            remove_mode = False

        if remove_mode:
            if is_double:
                # If the cursor is over a marker, prefer that exact marker; otherwise fall back to nearest
                target_idx = None
                try:
                    # ipyleaflet doesn't expose hovered marker, so emulate by finding a marker at the same location
                    for mk, idx0 in marker_objects:
                        loc = mk.location
                        if abs(float(loc[0]) - float(latlng[0])) < 1e-6 and abs(float(loc[1]) - float(latlng[1])) < 1e-6:
                            target_idx = getattr(mk, 'row_index', idx0)
                            break
                except Exception:
                    target_idx = None
                if target_idx is None:
                    # fallback: nearest
                    def _nearest_idx(lat, lng):
                        nearest = None
                        best = 1e-9
                        for i, row in log_df.iterrows():
                            c = row.get('coords')
                            if c is None or len(c) != 2:
                                continue
                            d = (float(c[0]) - float(lat))**2 + (float(c[1]) - float(lng))**2
                            if d < best:
                                best = d
                                nearest = i
                        return nearest
                    target_idx = _nearest_idx(latlng[0], latlng[1])
                if target_idx is not None:
                    log_df = log_df.drop(target_idx).reset_index(drop=True)
                    refresh_all(refresh_map_flag=True)
            # if single click in remove mode: do nothing
            return
        else:
            # Toggle OFF: add marker on single click
            current_time = now_str()
            new_row = {
                # Identity
                "symbol": chosen_symbol,
                "entry_id": str(uuid.uuid4()),
                # Contact identity (generate immediately so Contact-centric filters include this row)
                "contact_uuid": str(uuid.uuid4()),
                "contact_ref": generate_contact_ref(),
                "contact_short": "",  # derived below after ref exists
                "contact_source_id": "",
                "contact_status": "Active",
                "affiliation": affiliation_sel.v_model if 'affiliation_sel' in globals() else '',
                "unit_identity": unit_identity_tf.v_model if 'unit_identity_tf' in globals() else '',
                "report_type": report_type_sel.v_model if 'report_type_sel' in globals() else '',
                "thematic_tags": thematic_tags_tf.v_model if 'thematic_tags_tf' in globals() else '',
                # Time
                "when": current_time,
                "reported_at": current_time,
                "staleness_minutes": '',
                # Location
                "coords": list(latlng),
                "lat": float(latlng[0]),
                "lon": float(latlng[1]),
                "mgrs": '',
                "mgrs_precision_m": '',
                "geo_source": '',
                "location_text": '',
                # Content
                "size": '',
                "activity": '',
                "equipment": '',
                "time_observed": '',
                "description": '',
                "attachments_ref": '',
                # Evaluation
                "source_reliability": '',
                "information_credibility": '',
                "classification": '',
                "handling_instructions": '',
                "analyst_assessment": '',
                "analyst_confidence": '',
                # Relevance (defaults must NOT copy prior UI state)
                "pir_id": '',
                "sir_id": '',
                "ccir_flag": 'False',
                "priority": 'Routine',
                "critical_manual": 'False',
                "critical_int": 'False',
                # Provenance
                "reporting_unit": '',
                "reporter_type": '',
                "reporter_id": '',
                "collection_method": '',
                # Workflow
                "status": status_sel.v_model if 'status_sel' in globals() else '',
                "action_taken": action_taken_tf.v_model if 'action_taken_tf' in globals() else '',
                "dissemination_list": dissemination_list_tf.v_model if 'dissemination_list_tf' in globals() else '',
                "duplicates_of": '',
                "created_by": '',
                "reviewed_by": '',
                "last_updated_dtg": now_str(),
                # Legacy / misc
                "source_reference": dlg_src_ref.v_model if 'dlg_src_ref' in globals() else ''
            }
            # Derive contact_short from ref
            try:
                new_row["contact_short"] = derive_contact_short(new_row.get("contact_ref", ""))
            except Exception:
                new_row["contact_short"] = ""
            # append row and open dialog to edit details
            log_df.loc[len(log_df)] = new_row
            try:
                _append_version(new_row['entry_id'], op='create', payload=_canonical_payload(pd.Series(new_row)))
            except Exception:
                pass
            _open_entry_dialog(new_row, len(log_df) - 1)
            refresh_all(refresh_map_flag=True)
m.on_interaction(handle_click)
# Also listen for explicit double-click events if available
def handle_dblclick(**kwargs):
    if kwargs.get('type') == 'dblclick':
        try:
            remove = bool(remove_switch.v_model)
        except NameError:
            remove = True
        if not remove:
            return
        latlng = kwargs['coordinates']
        # Prefer exact marker under cursor if any
        target_idx = None
        try:
            for mk, idx0 in marker_objects:
                loc = mk.location
                if abs(float(loc[0]) - float(latlng[0])) < 1e-6 and abs(float(loc[1]) - float(latlng[1])) < 1e-6:
                    target_idx = getattr(mk, 'row_index', idx0)
                    break
        except Exception:
            target_idx = None
        if target_idx is None:
            def _nearest_idx(lat, lng):
                nearest = None
                best = 1e-9
                for i, row in log_df.iterrows():
                    c = row.get('coords')
                    if c is None or len(c) != 2:
                        continue
                    d = (float(c[0]) - float(lat))**2 + (float(c[1]) - float(lng))**2
                    if d < best:
                        best = d
                        nearest = i
                return nearest
            target_idx = _nearest_idx(latlng[0], latlng[1])
        if target_idx is not None:
            global log_df
            log_df = log_df.drop(target_idx).reset_index(drop=True)
            refresh_all(refresh_map_flag=True)
m.on_interaction(handle_dblclick)
def on_zoom_change(event):
    # On zoom: refresh markers and disable Enhance Location to minimize compute
    refresh_map()
    try:
        if enhance_location_switch.v_model:
            enhance_location_switch.v_model = False
        _clear_location_circles()
    except Exception:
        pass
m.observe(on_zoom_change, names='zoom')

filter_row = widgets.HBox([])

# ---- Vuetify controls ----
current_symbol_v = [symbol_options[0]]
v_symbol_select = v.Select(
    items=symbol_options,
    v_model=symbol_options[0],
    label='Nato Type',
    filled=True,
    dense=True,
    class_='ma-2',
    style_='min-width:220px; max-width:260px;'
)
# Observe v_model directly for reliable updates in Voila
def _on_v_symbol_change(change):
    current_symbol_v[0] = change['new']
    # update the small preview icon next to the field
    try:
        symbol_preview.attributes = {
            'src': svg_to_datauri(SVGs[current_symbol_v[0]]),
            'height': '30',
            'style': 'margin-left:8px; margin-right:8px; vertical-align:middle;'
        }
    except Exception:
        pass
v_symbol_select.observe(_on_v_symbol_change, names=['v_model'])

# Small preview icon placed as a separate element next to the field (avoids menu overlap)
symbol_preview = v.Html(
    tag='img',
    attributes={
        'src': svg_to_datauri(SVGs[current_symbol_v[0]]),
        'height': '30',
        'style': 'margin-left:8px; margin-right:8px; vertical-align:middle;'
    },
    class_='align-self-center'
)
remove_switch = v.Switch(label='Double click to delete Contact', v_model=False, class_='ma-1', style_='margin-top:2px; margin-bottom:0;')

# Basemap selector (remove custom fullscreen button in favor of Leaflet control)
base_items = list(base_layers.keys())
base_select = v.Select(items=base_items, v_model=base_items[0], label='Base layer', filled=True, dense=True, class_='ma-2', style_='min-width:180px; max-width:220px;')

# Enhance Location control (toggle only)
enhance_location_switch = v.Switch(label='Enhance Location', v_model=False, class_='ma-1', style_='margin-top:2px; margin-bottom:0;')
# Placeholder toggle for future filtering logic
filter_critical_switch = v.Switch(label='Filter for Critical INT', v_model=False, class_='ma-1', style_='margin-top:2px; margin-bottom:0;')

def _on_enhance_toggle(change):
    try:
        if change.get('name') == 'v_model':
            if change['new']:
                enhance_location_overlays()
            else:
                _clear_location_circles()
    except Exception:
        pass
enhance_location_switch.observe(_on_enhance_toggle, names=['v_model'])

def _on_base_change(change):
    name = change['new']
    new_layer = base_layers.get(name)
    if new_layer is None:
        return
    # Swap base layer while keeping any other layers (markers etc.)
    other_layers = [ly for ly in list(m.layers)[1:]] if len(m.layers) > 0 else []
    m.layers = tuple([new_layer] + other_layers)
    current_base_layer[0] = new_layer
base_select.observe(_on_base_change, names=['v_model'])

map_card_text = [None]

# ---- Entry Dialog (Add/Edit details after placing a marker) ----
# Shared fields for the modal
dlg_symbol = v.Select(items=symbol_options, v_model=symbol_options[0], label='Nato Type', filled=True, dense=True, class_='ma-2', color='#111827')
dlg_when = v.TextField(v_model='', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')
dlg_coords = v.TextField(v_model='', label='', filled=True, dense=True, class_='ma-2 nato-input', readonly=True, color='#111827')
dlg_reported_at = v.TextField(v_model='', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')
dlg_reporter_id = v.TextField(v_model='', label='', filled=True, dense=True, class_='nato-input', hide_details=True, color='#111827')
dlg_reporter_type = v.Select(items=['HUMINT','UAS FMV','EW','Patrol','OSINT','Partner','Other'], v_model='Patrol', label='', filled=True, dense=True, class_='nato-input', hide_details=True, color='#111827')
dlg_src_rel = v.Select(items=['A','B','C','D','E','F'], v_model='C', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')
dlg_info_conf = v.Select(items=['1','2','3','4','5','6'], v_model='3', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')
dlg_description = v.Textarea(v_model='', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')
dlg_src_ref = v.TextField(v_model='', label='', filled=True, dense=True, class_='ma-2 nato-input', color='#111827')

# --- Improved schema fields ---
affiliation_sel = v.Select(items=['Friendly','Hostile','Neutral','Unknown','Civilian'], v_model='Friendly', label='', filled=True, dense=True, class_='ma-2')
unit_identity_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
report_type_sel = v.Select(items=['SPOT','INTREP','SIGINT cut','HUMINT contact','OSINT tip','Other'], v_model='SPOT', label='', dense=True, filled=True, class_='ma-2')
thematic_tags_tf = v.TextField(v_model='', label='(comma separated)', dense=True, filled=True, class_='ma-2 nato-input')

mgrs_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
mgrs_prec_sel = v.Select(items=['10','100','1000'], v_model='100', label='', dense=True, filled=True, class_='ma-2')
geo_source_sel = v.Select(items=['GPS','Map estimate','From media','Other'], v_model='GPS', label='', dense=True, filled=True, class_='ma-2')
location_text_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')

# --- Contact section widgets ---
contact_uuid_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input', readonly=True, hide_details=True)
contact_ref_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input', hide_details=True)
contact_short_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input', hide_details=True)
contact_source_id_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input', hide_details=True)
contact_status_sel = v.Select(items=['Active','Closed'], v_model='Active', label='', dense=True, filled=True, class_='ma-2')
contact_close_reason_sel = v.Select(items=[
    'Confirmed destroyed/neutralised',
    'Departed AO with no expectation of return',
    'Reclassified as invalid (false positive)',
    'Merged into another contact',
    'No updates within staleness threshold (doctrine-based)'
], v_model='', label='', dense=True, filled=True, class_='ma-2')
contact_close_dtg_tf = v.TextField(v_model='', label='YYYY-MM-DD:HH:MM:SS', dense=True, filled=True, class_='ma-2 nato-input', hide_details=True)

size_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
activity_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
equipment_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
time_observed_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
attachments_ref_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')

classification_sel = v.Select(items=['OFFICIAL','NATO RESTRICTED','SECRET','TOP SECRET'], v_model='OFFICIAL', label='', dense=True, filled=True, class_='ma-2')
handling_instr_sel = v.Select(items=['REL TO','NOFORN','ORCON','None'], v_model='None', label='', dense=True, filled=True, class_='ma-2')
analyst_assess_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='ma-2 nato-input')
analyst_conf_sel = v.Select(items=['Low','Medium','High'], v_model='Medium', label='', dense=True, filled=True, class_='ma-2')

pir_id_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
sir_id_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
ccir_flag_sw = v.Switch(v_model=False, label='', class_='ma-1', style_='margin-top:6px;')
priority_sel = v.Select(items=['Routine','Priority','Immediate'], v_model='Routine', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
# Manual override control; when ON, user can set Critical Int state manually via dropdown
critical_manual_sw = v.Switch(v_model=False, label='Manually Derived', class_='ma-2')
# Dropdown representing Critical Int Status; values stored as strings 'True'/'False' for DF compatibility
critical_int_sel = v.Select(
    items=[{'text': '🔴 Critical', 'value': 'True'}, {'text': '⚪ Not Critical', 'value': 'False'}],
    v_model='False', label='', dense=True, filled=True, class_='ma-2', style_='min-width:180px;'
)

reporting_unit_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
collection_method_sel = v.Select(items=['FMV','intercept','debrief','patrol log','social media','other'], v_model='patrol log', label='', dense=True, filled=True, class_='nato-input', hide_details=True)

status_sel = v.Select(items=['New','Triaged','Fused','Disseminated','Closed'], v_model='New', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
action_taken_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)
dissemination_list_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)

# UID controls (Workflow / QA)
ict_uid_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', readonly=True, hide_details=True)
short_uid_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', readonly=True, hide_details=True)
source_uid_tf = v.TextField(v_model='', label='', dense=True, filled=True, class_='nato-input', hide_details=True)

_dlg_row_index = [-1]

def _open_entry_dialog(prefill: dict, row_index: int):
    _dlg_row_index[0] = row_index
    # Helper to coerce various truthy strings/bools to bool
    def _parse_bool(v):
        if isinstance(v, bool):
            return v
        s = str(v).strip().lower()
        return s in ('true', '1', 'yes', 'on')

    dlg_symbol.v_model = prefill.get('symbol', symbol_options[0])
    dlg_when.v_model = prefill.get('when', '')
    dlg_coords.v_model = str(prefill.get('coords', ''))
    dlg_reported_at.v_model = prefill.get('reported_at', '')
    dlg_reporter_id.v_model = prefill.get('reporter_id', '')
    dlg_reporter_type.v_model = prefill.get('reporter_type', '')
    dlg_src_rel.v_model = prefill.get('source_reliability', '')
    # Maintain dialog control id but align data name to information_credibility
    dlg_info_conf.v_model = prefill.get('information_credibility', prefill.get('info_confidence', '')) or '3'
    dlg_description.v_model = prefill.get('description', '')
    dlg_src_ref.v_model = prefill.get('source_reference', '')
    # Identity extras
    # Preserve user-chosen Affiliation coming from toolbar; otherwise default to 'Friendly'
    try: affiliation_sel.v_model = prefill.get('affiliation', affiliation_sel.v_model or 'Friendly')
    except Exception: pass
    try: unit_identity_tf.v_model = prefill.get('unit_identity', '')
    except Exception: pass
    try: report_type_sel.v_model = prefill.get('report_type', report_type_sel.v_model or 'SPOT')
    except Exception: pass
    try: thematic_tags_tf.v_model = prefill.get('thematic_tags', '')
    except Exception: pass
    # Location fields
    try:
        mgrs_tf.v_model = prefill.get('mgrs', '')
    except Exception:
        pass
    try:
        val = prefill.get('mgrs_precision_m', '')
        mgrs_prec_sel.v_model = str(val) if val is not None and val != '' else (mgrs_prec_sel.v_model or '100')
    except Exception:
        pass
    try:
        geo_source_sel.v_model = prefill.get('geo_source', geo_source_sel.v_model or 'GPS')
    except Exception:
        pass
    try:
        location_text_tf.v_model = prefill.get('location_text', '')
    except Exception:
        pass
    # Content defaults
    try: size_tf.v_model = prefill.get('size', '')
    except Exception: pass
    try: activity_tf.v_model = prefill.get('activity', '')
    except Exception: pass
    try: equipment_tf.v_model = prefill.get('equipment', '')
    except Exception: pass
    try: time_observed_tf.v_model = prefill.get('time_observed', '')
    except Exception: pass
    try: attachments_ref_tf.v_model = prefill.get('attachments_ref', '')
    except Exception: pass
    # Evaluation defaults
    try: classification_sel.v_model = prefill.get('classification', classification_sel.v_model or 'OFFICIAL')
    except Exception: pass
    try: handling_instr_sel.v_model = prefill.get('handling_instructions', handling_instr_sel.v_model or 'None')
    except Exception: pass
    try: analyst_assess_tf.v_model = prefill.get('analyst_assessment', '')
    except Exception: pass
    try: analyst_conf_sel.v_model = prefill.get('analyst_confidence', analyst_conf_sel.v_model or 'Medium')
    except Exception: pass
    # Relevance / Tasking defaults
    try: pir_id_tf.v_model = prefill.get('pir_id', '')
    except Exception: pass
    try: sir_id_tf.v_model = prefill.get('sir_id', '')
    except Exception: pass
    try: ccir_flag_sw.v_model = _parse_bool(prefill.get('ccir_flag', False))
    except Exception: pass
    try: priority_sel.v_model = prefill.get('priority', 'Routine')
    except Exception: pass
    try: critical_manual_sw.v_model = _parse_bool(prefill.get('critical_manual', False))
    except Exception: pass
    # Derive or pull Critical Int value for display
    try:
        auto_value = 'True' if (ccir_flag_sw.v_model or ((pir_id_tf.v_model or sir_id_tf.v_model) and priority_sel.v_model in ('Immediate','Priority'))) else 'False'
        if critical_manual_sw.v_model:
            # Respect existing manual choice; keep dropdown as-is if provided; else default to auto
            provided = prefill.get('critical_int', None)
            critical_int_sel.v_model = 'True' if _parse_bool(provided) else (critical_int_sel.v_model or auto_value)
        else:
            critical_int_sel.v_model = auto_value
    except Exception:
        pass
    # Sanitize helper for text-like fields that might contain NaN/NaT
    def _clean_text(val: Any) -> str:
        try:
            if val is None:
                return ''
            s = str(val)
            return '' if s.strip().lower() in ('nan', 'nat', 'none') else s
        except Exception:
            return ''

    # Populate UID controls from prefill / df row
    try:
        ict_uid_tf.v_model = _clean_text(prefill.get('ict_uid', ''))
    except Exception:
        pass
    try:
        short_uid_tf.v_model = prefill.get('short_uid', '')
    except Exception:
        pass
    try:
        source_uid_tf.v_model = prefill.get('source_uid', '')
    except Exception:
        pass
    # UID fields
    try:
        # Access fields created in the Workflow panel
        for panel in entry_panels.children:
            pass
    except Exception:
        pass
    # Ensure Contact identity exists on first open (generate if missing/NaN) and write into log_df
    try:
        # Contact UUID
        cu_val = _clean_text(prefill.get('contact_uuid', ''))
        if not cu_val and row_index in log_df.index:
            cu_val = _clean_text(log_df.loc[row_index].get('contact_uuid', ''))
        if not cu_val:
            cu_val = str(uuid.uuid4())
        contact_uuid_tf.v_model = cu_val
        try:
            log_df.at[row_index, 'contact_uuid'] = cu_val
        except Exception:
            pass
        # Contact Ref + Short (generate if missing)
        cr_val = _clean_text(prefill.get('contact_ref', '')) or _clean_text(log_df.loc[row_index].get('contact_ref', '') if row_index in log_df.index else '')
        if not cr_val:
            cr_val = generate_contact_ref()
        contact_ref_tf.v_model = cr_val
        contact_short_tf.v_model = derive_contact_short(cr_val)
        try:
            log_df.at[row_index, 'contact_ref'] = cr_val
            log_df.at[row_index, 'contact_short'] = derive_contact_short(cr_val)
        except Exception:
            pass
    except Exception:
        pass

    # Auto-create first Serial (ICT UID) when opening a new contact/row lacking ict_uid
    try:
        current_ict = _clean_text(prefill.get('ict_uid', '')) or (_clean_text(log_df.loc[row_index].get('ict_uid', '')) if row_index in log_df.index else '')
        if not current_ict:
            new_ict = generate_ict_uid()
            new_short = derive_short_uid(new_ict)
            ict_uid_tf.v_model = new_ict
            short_uid_tf.v_model = new_short
            try:
                log_df.at[row_index, 'ict_uid'] = new_ict
                log_df.at[row_index, 'short_uid'] = new_short
            except Exception:
                pass
    except Exception:
        pass

    # Populate Serial selector for this contact_uuid and wire handlers
    try:
        contact_id = contact_uuid_tf.v_model or (str(log_df.loc[row_index].get('contact_uuid','')) if row_index in log_df.index else '')
        options = []
        if contact_id:
            seen = set()
            for _, r in log_df[log_df['contact_uuid'] == contact_id].iterrows():
                ict = str(r.get('ict_uid',''))
                if not ict or ict in seen:
                    continue
                seen.add(ict)
                short = str(r.get('short_uid',''))
                label = f"{ict} ({short})" if short else ict
                options.append({'text': label, 'value': ict})
        # Append inline + New Serial option
        options.append({'text': '+ New Serial', 'value': '__NEW_SERIAL__'})
        serial_selector.items = options
        serial_selector.v_model = ict_uid_tf.v_model or (str(log_df.loc[row_index].get('ict_uid','')) if row_index in log_df.index else '')
    except Exception:
        try:
            serial_selector.items = []
            serial_selector.v_model = ''
        except Exception:
            pass

    # Update the serial count badge in the Contact header
    try:
        if contact_id:
            count = len({str(r.get('ict_uid','')) for _, r in log_df[log_df['contact_uuid'] == contact_id].iterrows() if str(r.get('ict_uid',''))})
            contact_serial_count_badge.children = [f"({count} serials)"]
        else:
            contact_serial_count_badge.children = ['']
    except Exception:
        try:
            contact_serial_count_badge.children = ['']
        except Exception:
            pass

    def _on_serial_change(widget, event, data, _contact_id=contact_id):
        # ipyvuetify may pass either the value or the whole item or the text label
        if isinstance(data, dict):
            val = data.get('value') or data.get('v') or data.get('text') or ''
        else:
            val = data
        if not val:
            return
        # Handle inline new-serial option (match by value or label)
        if str(val) == '__NEW_SERIAL__' or str(data).strip().lower().startswith('+ new serial'):
            try:
                _on_new_serial(widget, event, data, _contact_id=_contact_id)
                _toast('New Serial created')
            except Exception:
                pass
            return
        try:
            rows = log_df[(log_df['contact_uuid'] == _contact_id) & (log_df['ict_uid'] == str(val))]
            if not rows.empty:
                idx = int(rows.index[-1])
                _open_entry_dialog(log_df.loc[idx].to_dict(), idx)
        except Exception:
            pass
    try:
        serial_selector.on_event('change', _on_serial_change)
    except Exception:
        pass

    def _on_new_serial(widget, event, data, _contact_id=contact_id):
        global log_df
        try:
            # Preserve current expanded panel index to avoid visual jump
            try:
                current_panel = entry_panels.v_model
            except Exception:
                current_panel = None
            base = log_df.loc[row_index] if row_index in log_df.index else pd.Series({})
            new_ict = generate_ict_uid()
            coords_val = list(base.get('coords', [])) if isinstance(base.get('coords', []), (list, tuple)) else []
            lat_val = float(base.get('lat')) if 'lat' in base and pd.notnull(base.get('lat')) else (coords_val[0] if len(coords_val)==2 else None)
            lon_val = float(base.get('lon')) if 'lon' in base and pd.notnull(base.get('lon')) else (coords_val[1] if len(coords_val)==2 else None)
            new_row = base.to_dict()
            new_row.update({
                'entry_id': str(uuid.uuid4()),
                'ict_uid': new_ict,
                'short_uid': derive_short_uid(new_ict),
                'source_uid': '',
                'coords': coords_val if coords_val else None,
                'lat': lat_val if lat_val is not None else None,
                'lon': lon_val if lon_val is not None else None,
                'status': 'New',
                'action_taken': '',
                'dissemination_list': '',
                'description': '',
                'attachments_ref': '',
            })
            new_row['contact_uuid'] = _contact_id or new_row.get('contact_uuid') or str(uuid.uuid4())
            # Clean contact_ref/short if NaN
            try:
                if not new_row.get('contact_ref') or str(new_row.get('contact_ref')).strip().lower() in ('nan','nat','none'):
                    new_row['contact_ref'] = generate_contact_ref()
                new_row['contact_short'] = derive_contact_short(new_row.get('contact_ref',''))
            except Exception:
                pass
            log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)
            try:
                _append_version(new_row['entry_id'], op='create', payload=_canonical_payload(pd.Series(new_row)))
            except Exception:
                pass
            # Refresh the selector to include the new ICT UID and select it
            try:
                items = list(serial_selector.items or [])
            except Exception:
                items = []
            try:
                label = f"{new_row['ict_uid']} ({new_row.get('short_uid','')})" if new_row.get('short_uid') else new_row['ict_uid']
                items = [it for it in items if not (isinstance(it, dict) and it.get('value') == new_row['ict_uid'])]
                items.append({'text': label, 'value': new_row['ict_uid']})
                # Maintain '+ New Serial' option at end
                items = [it for it in items if not (isinstance(it, dict) and it.get('value') == '__NEW_SERIAL__')] + [{'text': '+ New Serial', 'value': '__NEW_SERIAL__'}]
                serial_selector.items = items
                serial_selector.v_model = new_row['ict_uid']
            except Exception:
                pass
            _open_entry_dialog(new_row, len(log_df)-1)
            # Restore panel index to avoid scroll-to-top behavior
            try:
                if current_panel is not None:
                    entry_panels.v_model = current_panel
            except Exception:
                pass
            _toast('New Serial created')
        except Exception:
            pass
    # new_serial_btn removed; creation handled via inline select option

    # Prefill Contact section
    try:
        contact_uuid_tf.v_model = prefill.get('contact_uuid', contact_uuid_tf.v_model or str(uuid.uuid4()))
        # If no contact_ref, auto-generate one
        contact_ref_tf.v_model = prefill.get('contact_ref', contact_ref_tf.v_model or generate_contact_ref())
        # Derive contact_short from ref
        contact_short_tf.v_model = prefill.get('contact_short', derive_contact_short(contact_ref_tf.v_model))
        contact_source_id_tf.v_model = prefill.get('contact_source_id', '')
        contact_status_sel.v_model = prefill.get('contact_status', 'Active')
        contact_close_reason_sel.v_model = prefill.get('contact_close_reason', '')
        contact_close_dtg_tf.v_model = prefill.get('contact_close_dtg', '')
    except Exception:
        pass
    # Ensure top-of-modal on open by rebuilding the dialog content
    try:
        _rebuild_entry_dialog()
    except Exception:
        pass
    entry_dialog.v_model = True
    # Update dialog title to show NATO type and icon, and make it reactive to NATO Type changes
    try:
        def _render_title(name: str):
            # Add critical badge if current row being edited is marked critical
            try:
                row = log_df.loc[_dlg_row_index[0]] if _dlg_row_index[0] in log_df.index else None
                is_crit = str(row.get('critical_int','')).lower() in ('true','1','yes','on') if row is not None else False
            except Exception:
                is_crit = False
            base_svg = SVGs.get(name, SVGs[symbol_options[0]])
            icon_badged = base_svg.replace('</svg>', "<circle cx='12' cy='12' r='12' fill='#dc2626'/></svg>") if is_crit else base_svg
            icon_url0 = svg_to_datauri(icon_badged)
            return v.Html(tag='div', children=[
                v.Html(tag='img', attributes={'src': icon_url0, 'height': '20'}),
                v.Html(tag='span', children=[str(name)])
            ], style_='display:flex;align-items:center;gap:10px;')

        nato_name = dlg_symbol.v_model or symbol_options[0]
        entry_dialog_content.children[0].children = [_render_title(nato_name)]

        # Bind change handler to update title when NATO type changes
        def _on_nato_change(widget=None, event=None, data=None):
            try:
                new_name = data if data is not None else (dlg_symbol.v_model or symbol_options[0])
                entry_dialog_content.children[0].children = [_render_title(new_name)]
            except Exception:
                pass
        try:
            dlg_symbol.on_event('change', _on_nato_change)
        except Exception:
            pass
    except Exception:
        pass

    # Reactive handlers: when key fields change and manual override is off, recompute Critical Int
    def _maybe_auto_update(*args, **kwargs):
        try:
            if not bool(critical_manual_sw.v_model):
                is_ccir = bool(ccir_flag_sw.v_model)
                has_pir_sir = bool(pir_id_tf.v_model) or bool(sir_id_tf.v_model)
                is_priority = str(priority_sel.v_model) in ('Immediate','Priority')
                derived = 'True' if (is_ccir or (has_pir_sir and is_priority)) else 'False'
                critical_int_sel.v_model = derived
        except Exception:
            pass

    # Bind change listeners
    try: ccir_flag_sw.observe(lambda c: _maybe_auto_update(), names=['v_model'])
    except Exception: pass
    try: pir_id_tf.on_event('input', lambda *a: _maybe_auto_update())
    except Exception: pass
    try: sir_id_tf.on_event('input', lambda *a: _maybe_auto_update())
    except Exception: pass
    try: priority_sel.observe(lambda c: _maybe_auto_update(), names=['v_model'])
    except Exception: pass

def _submit_entry_dialog(widget=None, *args):
    global log_df
    try:
        idx = _dlg_row_index[0]
        if idx >= 0 and idx in log_df.index:
            # Derive Critical Int unless manual override is on
            def _derive_critical(pir: str, sir: str, ccir: str, priority: str, manual_override: bool, current_value: str) -> bool:
                try:
                    if str(ccir).lower() in ('true','1','yes','on'):
                        return True
                    if (pir or sir) and str(priority) in ('Immediate','Priority'):
                        return True
                    if manual_override:
                        return str(current_value).lower() in ('true','1','yes','on')
                except Exception:
                    pass
                return False
            log_df.at[idx, 'symbol'] = dlg_symbol.v_model
            log_df.at[idx, 'when'] = dlg_when.v_model
            log_df.at[idx, 'reported_at'] = dlg_reported_at.v_model
            log_df.at[idx, 'reporter_id'] = dlg_reporter_id.v_model
            log_df.at[idx, 'reporter_type'] = dlg_reporter_type.v_model
            log_df.at[idx, 'source_reliability'] = dlg_src_rel.v_model
            log_df.at[idx, 'information_credibility'] = dlg_info_conf.v_model
            log_df.at[idx, 'description'] = dlg_description.v_model
            log_df.at[idx, 'source_reference'] = dlg_src_ref.v_model
            # Identity
            log_df.at[idx, 'affiliation'] = affiliation_sel.v_model
            log_df.at[idx, 'unit_identity'] = unit_identity_tf.v_model
            log_df.at[idx, 'report_type'] = report_type_sel.v_model
            log_df.at[idx, 'thematic_tags'] = thematic_tags_tf.v_model
            # Content
            log_df.at[idx, 'size'] = size_tf.v_model
            log_df.at[idx, 'activity'] = activity_tf.v_model
            log_df.at[idx, 'equipment'] = equipment_tf.v_model
            log_df.at[idx, 'time_observed'] = time_observed_tf.v_model
            log_df.at[idx, 'attachments_ref'] = attachments_ref_tf.v_model
            # Evaluation
            log_df.at[idx, 'classification'] = classification_sel.v_model
            log_df.at[idx, 'handling_instructions'] = handling_instr_sel.v_model
            log_df.at[idx, 'analyst_assessment'] = analyst_assess_tf.v_model
            log_df.at[idx, 'analyst_confidence'] = analyst_conf_sel.v_model
            # Relevance / Tasking
            log_df.at[idx, 'pir_id'] = pir_id_tf.v_model
            log_df.at[idx, 'sir_id'] = sir_id_tf.v_model
            log_df.at[idx, 'ccir_flag'] = str(ccir_flag_sw.v_model)
            log_df.at[idx, 'priority'] = priority_sel.v_model
            log_df.at[idx, 'critical_manual'] = str(critical_manual_sw.v_model)
            # Apply derivation logic
            derived_val = _derive_critical(
                pir_id_tf.v_model,
                sir_id_tf.v_model,
                str(ccir_flag_sw.v_model),
                priority_sel.v_model,
                bool(critical_manual_sw.v_model),
                critical_int_sel.v_model
            )
            critical_int_sel.v_model = 'True' if derived_val else 'False'
            log_df.at[idx, 'critical_int'] = 'True' if derived_val else 'False'
            # Provenance
            log_df.at[idx, 'reporting_unit'] = reporting_unit_tf.v_model
            log_df.at[idx, 'collection_method'] = collection_method_sel.v_model
            # Workflow / QA
            log_df.at[idx, 'status'] = status_sel.v_model
            log_df.at[idx, 'action_taken'] = action_taken_tf.v_model
            log_df.at[idx, 'dissemination_list'] = dissemination_list_tf.v_model
            # Ensure entry identity
            try:
                if not str(log_df.at[idx, 'entry_id']):
                    log_df.at[idx, 'entry_id'] = str(uuid.uuid4())
            except Exception:
                log_df.at[idx, 'entry_id'] = str(uuid.uuid4())
            # UIDs: generate/persist
            try:
                existing_ict = str(log_df.at[idx, 'ict_uid']) if ('ict_uid' in log_df.columns and pd.notnull(log_df.at[idx, 'ict_uid'])) else ''
            except Exception:
                existing_ict = ''
            if not existing_ict:
                existing_ict = generate_ict_uid()
            log_df.at[idx, 'ict_uid'] = existing_ict
            log_df.at[idx, 'short_uid'] = derive_short_uid(existing_ict)
            # source_uid from dialog input (nullable)
            try:
                log_df.at[idx, 'source_uid'] = source_uid_tf.v_model
            except Exception:
                pass
            # Append a version snapshot (audit trail) regardless of which fields changed
            try:
                payload = _canonical_payload(log_df.loc[idx])
                entry_id = str(payload.get('entry_id') or '')
                if not entry_id:
                    entry_id = str(uuid.uuid4())
                    log_df.at[idx, 'entry_id'] = entry_id
                    payload['entry_id'] = entry_id
                _append_version(entry_id, op='update', payload=payload)
            except Exception:
                pass

            # Ensure the new/updated row is visible in both map and table immediately
            try:
                refresh_all(refresh_map_flag=True)
            except Exception:
                pass

            # Refresh serial selector items/labels in case short_uid changed
            try:
                cu = str(log_df.at[idx, 'contact_uuid']) if 'contact_uuid' in log_df.columns else ''
                if cu:
                    items = []
                    seen = set()
                    for _, r in log_df[log_df['contact_uuid'] == cu].iterrows():
                        ict = str(r.get('ict_uid',''))
                        if not ict or ict in seen:
                            continue
                        seen.add(ict)
                        short = str(r.get('short_uid',''))
                        label = f"{ict} ({short})" if short else ict
                        items.append({'text': label, 'value': ict})
                    serial_selector.items = items
                    try:
                        serial_selector.v_model = str(log_df.at[idx, 'ict_uid'])
                    except Exception:
                        pass
            except Exception:
                pass
            # lat/lon derived from coords if available
            try:
                coord_pair = log_df.at[idx, 'coords']
                if isinstance(coord_pair, (list, tuple)) and len(coord_pair) == 2:
                    log_df.at[idx, 'lat'] = float(coord_pair[0])
                    log_df.at[idx, 'lon'] = float(coord_pair[1])
            except Exception:
                pass
            # Save Location fields used by Enhance Location overlays
            log_df.at[idx, 'mgrs'] = mgrs_tf.v_model
            log_df.at[idx, 'mgrs_precision_m'] = mgrs_prec_sel.v_model
            log_df.at[idx, 'geo_source'] = geo_source_sel.v_model
            log_df.at[idx, 'location_text'] = location_text_tf.v_model
            # Contact: enforce generation/derivation and validation
            try:
                if not str(log_df.at[idx, 'contact_uuid']):
                    log_df.at[idx, 'contact_uuid'] = str(uuid.uuid4())
            except Exception:
                log_df.at[idx, 'contact_uuid'] = str(uuid.uuid4())
            # Contact Ref: generate if missing; allow edits
            new_ref = contact_ref_tf.v_model or generate_contact_ref()
            log_df.at[idx, 'contact_ref'] = new_ref
            # Derive short from ref if possible
            log_df.at[idx, 'contact_short'] = derive_contact_short(new_ref)
            log_df.at[idx, 'contact_source_id'] = contact_source_id_tf.v_model
            # Status + close fields validation
            status_val = contact_status_sel.v_model or 'Active'
            if status_val == 'Closed':
                reason = contact_close_reason_sel.v_model or ''
                close_dtg = contact_close_dtg_tf.v_model or ''
                if not reason or not close_dtg:
                    # If invalid, keep status Active (do not allow Closed without required fields)
                    status_val = 'Active'
                else:
                    log_df.at[idx, 'contact_close_reason'] = reason
                    log_df.at[idx, 'contact_close_dtg'] = close_dtg
            log_df.at[idx, 'contact_status'] = status_val
    except Exception as e:
        print('Dialog save error:', e)
    entry_dialog.v_model = False
    refresh_all(refresh_map_flag=True)

def _close_entry_dialog(widget=None, *args):
    entry_dialog.v_model = False

toast_snackbar = v.Snackbar(v_model=False, bottom=True, right=True, timeout=3000, color='info')
def _toast(message: str):
    try:
        toast_snackbar.children = [str(message)]
        toast_snackbar.v_model = True
    except Exception:
        pass

entry_dialog = v.Dialog(v_model=False, max_width='1600px', persistent=True)
entry_dialog_content = v.Card(
    class_='pa-3',
    elevation=8,
    style_='background:#f9fafb;border-radius:10px;max-height:60vh;overflow:auto;',
    children=[
        # Title will be dynamically set to the selected NATO type and icon
        v.CardTitle(children=[v.Html(tag='div', children=[''], style_='display:flex;align-items:center;gap:10px;', class_='ict-entry-title')]),
        v.CardText(children=[
            (
                entry_panels := v.ExpansionPanels(v_model=0, accordion=True, children=[
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=[
                        'Contact',
                        (contact_serial_count_badge := v.Html(tag='span', children=[''], class_='ml-2', style_='font-size:12px;color:#6b7280;'))
                    ]),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Nato Type']), dlg_symbol]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Affiliation']), affiliation_sel]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Unit identity']), unit_identity_tf]),
                        ]),
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Report type']), report_type_sel]),
                            v.Col(cols=12, md=8, children=[v.Label(children=['Thematic tags']), thematic_tags_tf]),
                        ]),
                        # Contact subset
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Contact uuid']), contact_uuid_tf]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Contact Ref (CT-YYYYMMDD-SEQ)']), contact_ref_tf]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Contact Short (derived)']), contact_short_tf]),
                        ]),
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Contact Source_id']), contact_source_id_tf]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Contact Status']), contact_status_sel]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Close reason']), contact_close_reason_sel]),
                        ]),
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Close DTG']), contact_close_dtg_tf]),
                        ])
                    ])
                ]),
                # Serial Workflow moved above Time
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Serial Workflow']),
                    v.ExpansionPanelContent(children=[
                        # Serial selector row (ICT UID list for this Contact)
                        v.Row(children=[
                            v.Col(cols=12, md=6, children=[
                                v.Label(children=['Serial (ICT UID)'], class_='mb-1'),
                                (serial_selector := v.Select(items=[], v_model='', dense=True, filled=True, class_='nato-input', hide_details=True, item_text='text', item_value='value'))
                            ], class_='pa-1')
                        ], class_='ma-0'),
                        v.Row(children=[
                            v.Col(cols=12, md=3, children=[v.Label(children=['Status'], class_='mb-1'), status_sel], class_='pa-1'),
                            v.Col(cols=12, md=5, children=[v.Label(children=['Action taken'], class_='mb-1'), action_taken_tf], class_='pa-1'),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Dissemination list'], class_='mb-1'), dissemination_list_tf], class_='pa-1'),
                        ], class_='ma-0'),
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['ICT UID (auto)'], class_='mb-1'), ict_uid_tf], class_='pa-1'),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Short UID (auto)'], class_='mb-1'), short_uid_tf], class_='pa-1'),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Source UID'], class_='mb-1'), source_uid_tf], class_='pa-1'),
                        ], class_='ma-0')
                    ])
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Time']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=6, children=[v.Label(children=['event_dtg']), dlg_when]),
                            v.Col(cols=12, md=6, children=[v.Label(children=['report_dtg']), dlg_reported_at]),
                        ])
                    ])
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Location']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=6, children=[v.Label(children=['Coords (lat,lon)']), dlg_coords]),
                            v.Col(cols=12, md=6, children=[v.Label(children=['MGRS']), mgrs_tf]),
                        ]),
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Precision (m)']), mgrs_prec_sel]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Geo source']), geo_source_sel]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Location text']), location_text_tf]),
                        ])
                    ])
                ]),
                # Contact panel merged above
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Content (SALUTE)']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=3, children=[v.Label(children=['Size']), size_tf]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Activity']), activity_tf]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Equipment']), equipment_tf]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Time observed']), time_observed_tf]),
                        ]),
                        v.Row(children=[v.Col(cols=12, children=[v.Label(children=['Description']), dlg_description])]),
                        v.Row(children=[v.Col(cols=12, children=[v.Label(children=['Attachments ref']), attachments_ref_tf])])
                    ])
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Evaluation (5x5x5)']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=3, children=[v.Label(children=['Source reliability']), dlg_src_rel]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Information credibility']), dlg_info_conf]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Classification']), classification_sel]),
                            v.Col(cols=12, md=3, children=[v.Label(children=['Handling instructions']), handling_instr_sel]),
                        ]),
                        v.Row(children=[
                            v.Col(cols=12, md=8, children=[v.Label(children=['Analyst assessment']), analyst_assess_tf]),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Analyst confidence']), analyst_conf_sel]),
                        ])
                    ])
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Relevance / Tasking']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=3, children=[v.Label(children=['PIR id'], class_='mb-1'), pir_id_tf], class_='pa-1'),
                            v.Col(cols=12, md=3, children=[v.Label(children=['SIR id'], class_='mb-1'), sir_id_tf], class_='pa-1'),
                            v.Col(cols=12, md=2, children=[v.Label(children=['CCIR flag'], class_='mb-1'), ccir_flag_sw], class_='pa-1'),
                            v.Col(cols=12, md=2, children=[v.Label(children=['Priority'], class_='mb-1'), priority_sel], class_='pa-1'),
                        ], class_='ma-0'),
                        v.Row(children=[
                            v.Col(cols=12, md=3, children=[v.Label(children=['Critical Int Status'], class_='mb-1'), critical_int_sel], class_='pa-1'),
                            v.Col(cols=12, md=3, children=[critical_manual_sw], class_='pa-1'),
                            v.Col(cols=12, md=6, children=[v.Html(tag='div', children=[''])])
                        ], class_='ma-0')
                    ])
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=['Provenance']),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[
                            v.Col(cols=12, md=4, children=[v.Label(children=['Reporting unit'], class_='mb-1'), reporting_unit_tf], class_='pa-1'),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Reporter type'], class_='mb-1'), dlg_reporter_type], class_='pa-1'),
                            v.Col(cols=12, md=4, children=[v.Label(children=['Reporter id'], class_='mb-1'), dlg_reporter_id], class_='pa-1'),
                        ], class_='ma-0'),
                        v.Row(children=[
                            v.Col(cols=12, md=6, children=[v.Label(children=['Collection method'], class_='mb-1'), collection_method_sel], class_='pa-1')
                        ], class_='ma-0')
                    ])
                ])
            ])
            )
        ]),
        v.CardActions(children=[
            v.Spacer(),
            v.Btn(children=['History…'], class_='ict-btn ma-2'),
            v.Btn(children=['Delete'], class_='ict-btn ict-danger ma-2'),
            v.Btn(children=['Save'], class_='ict-btn ict-save ma-2'),
        ])
    ]
)
def _delete_current_row(widget=None, *args):
    global log_df
    try:
        idx = _dlg_row_index[0]
        if idx is not None and idx in log_df.index:
            log_df = log_df.drop(idx).reset_index(drop=True)
    except Exception:
        pass
    entry_dialog.v_model = False
    refresh_all(refresh_map_flag=True)

def _open_history_from_modal(widget=None, *args):
    try:
        idx = _dlg_row_index[0]
        entry_id = str(log_df.loc[idx].get('entry_id','')) if idx in log_df.index else ''
    except Exception:
        entry_id = ''
    open_history_dialog(entry_id)

entry_dialog_content.children[-1].children[1].on_event('click', _open_history_from_modal)
entry_dialog_content.children[-1].children[2].on_event('click', _delete_current_row)
entry_dialog_content.children[-1].children[3].on_event('click', _submit_entry_dialog)
entry_dialog.children = [entry_dialog_content]

# Helper: rebuild the dialog content to reset scroll position and rebind buttons
def _rebuild_entry_dialog():
    global entry_dialog_content
    try:
        entry_panels.v_model = 0
    except Exception:
        pass
    title_node = v.CardTitle(children=[v.Html(tag='div', children=[''], style_='display:flex;align-items:center;gap:10px;', class_='ict-entry-title')])
    body_node = v.CardText(children=[entry_panels])
    actions = v.CardActions(children=[
        v.Spacer(),
        v.Btn(children=['History…'], class_='ict-btn ma-2'),
        v.Btn(children=['Delete'], class_='ict-btn ict-danger ma-2'),
        v.Btn(children=['Save'], class_='ict-btn ict-save ma-2'),
    ])
    new_card = v.Card(class_='pa-3', elevation=8, style_='background:#f9fafb;border-radius:10px;max-height:80vh;overflow:auto;', children=[title_node, body_node, actions])
    # Rebind action events
    new_card.children[-1].children[1].on_event('click', _open_history_from_modal)
    new_card.children[-1].children[2].on_event('click', _delete_current_row)
    new_card.children[-1].children[3].on_event('click', _submit_entry_dialog)
    entry_dialog_content = new_card
    entry_dialog.children = [entry_dialog_content, toast_snackbar]

# Small helper to refresh the modal title icon/text from current state
def _refresh_modal_title():
    try:
        idx = _dlg_row_index[0]
        row = log_df.loc[idx] if (idx is not None and idx in log_df.index) else None
        symbol_name = dlg_symbol.v_model or (row.get('symbol', symbol_options[0]) if row is not None else symbol_options[0])
        is_crit = False
        try:
            is_crit = str(row.get('critical_int', '')).lower() in ('true','1','yes','on') if row is not None else False
        except Exception:
            is_crit = False
        base_svg = SVGs.get(symbol_name, SVGs[symbol_options[0]])
        icon_svg = wrap_with_red_dot(base_svg) if is_crit else base_svg
        icon_url = svg_to_datauri(icon_svg)
        entry_dialog_content.children[0].children = [
            v.Html(tag='div', children=[
                v.Html(tag='img', attributes={'src': icon_url, 'height': '20'}),
                v.Html(tag='span', children=[str(symbol_name)])
            ], style_='display:flex;align-items:center;gap:10px;')
        ]
    except Exception:
        pass

# ---- Delete Confirmation Dialog ----
_delete_target_idx = [-1]

def open_delete_confirm(idx: int):
    try:
        _delete_target_idx[0] = int(idx)
        delete_dialog.v_model = True
    except Exception:
        pass

def _cancel_delete(widget=None, *args):
    delete_dialog.v_model = False
    _delete_target_idx[0] = -1

def _confirm_delete(widget=None, *args):
    global log_df
    try:
        idx = _delete_target_idx[0]
        if idx is not None and idx in log_df.index:
            log_df = log_df.drop(idx).reset_index(drop=True)
    except Exception:
        pass
    delete_dialog.v_model = False
    _delete_target_idx[0] = -1
    refresh_all(refresh_map_flag=True)

delete_dialog = v.Dialog(v_model=False, max_width='420px')
delete_dialog_card = v.Card(children=[
    v.CardTitle(children=['Delete row?']),
    v.CardText(children=[v.Html(tag='div', children=['This action cannot be undone.'])]),
    v.CardActions(children=[
        v.Spacer(),
        v.Btn(children=['Cancel'], class_='ict-btn ict-cancel ma-2'),
        v.Btn(children=['Delete'], class_='ict-btn ict-danger ma-2')
    ])
])
delete_dialog.children = [delete_dialog_card]
delete_dialog_card.children[-1].children[0].on_event('click', _cancel_delete)
delete_dialog_card.children[-1].children[1].on_event('click', _confirm_delete)

# ---- Quick preview Menu (Option A) ----
quick_preview_open = [False]
quick_preview_title = v.Html(tag='div', children=[''])
quick_preview_lines = v.Html(tag='div', children=[''], style_='white-space:pre-wrap;')
quick_preview_edit_btn = v.Btn(children=['Edit…'], small=True, color='primary')
quick_history_btn = v.Btn(children=['History…'], small=True)

def _open_quick_preview(row_idx: int):
    try:
        if row_idx is None or row_idx not in log_df.index:
            quick_preview.v_model = False
            quick_preview_open[0] = False
            return
        row = log_df.loc[row_idx]
        is_crit = str(row.get('critical_int','')).lower() in ('true','1','yes','on')
        base_svg = SVGs.get(row.get('symbol',''), SVGs[symbol_options[0]])
        icon_url = svg_to_datauri(wrap_with_red_dot(base_svg)) if is_crit else svg_to_datauri(base_svg)
        name_display = str(row.get('reporter_id') or row.get('reporter_type') or '')
        quick_preview_title.children = [
            v.Row(children=[
                v.Html(tag='img', attributes={'src': icon_url, 'height': '20'}, style_='margin-right:8px;'),
                v.Html(tag='div', children=[str(row.get('symbol',''))])
            ], align='center')
        ]
        lines = []
        if name_display:
            lines.append(f"Name: {name_display}")
        ra = str(row.get('reported_at','') or '')
        if ra:
            lines.append(f"Reported At: {ra}")
        desc = str(row.get('description','') or '')
        if desc:
            lines.append(desc)
        quick_preview_lines.children = ['\n'.join(lines)]
        def _edit(widget, event, data, idx=row_idx):
            prefill = {
                'symbol': row.get('symbol',''),
                'when': row.get('when',''),
                'coords': row.get('coords', []),
                'reported_at': row.get('reported_at',''),
                'reporter_id': row.get('reporter_id',''),
                'reporter_type': row.get('reporter_type',''),
                'source_reliability': row.get('source_reliability',''),
                'information_credibility': row.get('information_credibility',''),
                'description': row.get('description',''),
             'source_reference': row.get('source_reference',''),
             # Location fields for Enhance Location overlays
             'mgrs': row.get('mgrs',''),
             'mgrs_precision_m': row.get('mgrs_precision_m',''),
             'geo_source': row.get('geo_source',''),
             'location_text': row.get('location_text',''),
             # UIDs
             'ict_uid': row.get('ict_uid',''),
             'short_uid': row.get('short_uid',''),
             'source_uid': row.get('source_uid',''),
            }
            quick_preview.v_model = False
            _open_entry_dialog(prefill, idx)
        quick_preview_edit_btn.on_event('click', _edit)
        quick_preview.v_model = True
        quick_preview_open[0] = True
    except Exception:
        pass

def _open_history_from_preview(widget=None, *args):
    try:
        # Use selected row from preview open
        # It is opened with row in scope; but we can read title lines or pass via closure
        # Fallback: open for first current row
        entry_id = ''
        try:
            entry_id = str(log_df.loc[editing_row_id[0]].get('entry_id','')) if editing_row_id[0] in log_df.index else ''
        except Exception:
            entry_id = ''
        if not entry_id:
            try:
                entry_id = str(filtered_sorted_df().iloc[0].get('entry_id',''))
            except Exception:
                entry_id = ''
        open_history_dialog(entry_id)
    except Exception:
        open_history_dialog('')

quick_preview_card = v.Card(children=[
    v.CardTitle(children=[quick_preview_title]),
    v.CardText(children=[quick_preview_lines]),
    v.CardActions(children=[v.Spacer(), quick_history_btn, quick_preview_edit_btn])
], elevation=6, class_='pa-2')
quick_history_btn.on_event('click', _open_history_from_preview)

quick_preview = v.Dialog(v_model=False, max_width='380px', persistent=False)
quick_preview.children = [quick_preview_card]

# Toolbar and tabs
from pathlib import Path
import base64
_base_dir = Path(os.getcwd())

_logo_path = _base_dir / 'army_intel_corp_logo.png'
print("Logo path:", _base_dir / 'army_intel_corp_logo.png')
try:
    print("Logo path:", _base_dir / 'army_intel_corp_logo.png')
    _logo_b64 = base64.b64encode(_logo_path.read_bytes()).decode('ascii')
    logo_url = f'data:image/jpeg;base64,{_logo_b64}'
except Exception:
    # Fallback to remote if local not found
    logo_url = 'https://upload.wikimedia.org/wikipedia/en/7/7e/Intelligence_Corps_%28United_Kingdom%29_cap_badge.jpg'

# Title styled with requested font, placed immediately to the left of the logo
title_html = v.Html(
    tag='div',
    children=['Intelligence Cell Dashboard'],
    style_='font-family:-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;'
           'font-weight:500; font-size:28px; height:72px; line-height:72px;'
           'display:flex; align-items:center; margin-right:12px; white-space:nowrap;'
)
toolbar = v.Toolbar(children=[
    v.Spacer(),
    title_html,
    v.Html(tag='img', attributes={'src': logo_url, 'height': '72', 'style': 'margin-right:12px;'}),
], elevation=0, class_='elevation-0', style_='box-shadow:none; background: transparent;')
tabs = v.Tabs(v_model=0)
tab_items = v.TabsItems(v_model=0)
# Build tabs dynamically to ensure switching works in Voila
tabs.children = [v.Tab(children=['Map']), v.Tab(children=['Log Table'])]
tab_items.children = [
    v.TabItem(children=[
        v.Card(elevation=6, style_='border-radius:10px; box-shadow: 0 4px 12px rgba(0,0,0,0.08);', children=[
            (
                map_card_text := v.CardText(children=[
                    v.Row(children=[
                        # Left column: NATO Type dropdown with the delete toggle directly beneath
                        v.Col(cols='auto', class_='d-flex flex-column justify-center', children=[
                            v_symbol_select,
                            remove_switch
                        ], style_='margin-right:8px; gap:4px;'),
                        # Symbol column centered vertically to the left stack
                        v.Col(cols='auto', class_='d-flex align-center justify-center', children=[symbol_preview], style_='margin:0 8px;'),
                        # Right area: toggles (left) and Contact Activity filter (right)
                        v.Col(cols='auto', class_='d-flex align-center', children=[
                            v.Row(children=[
                                v.Col(cols='auto', class_='d-flex flex-column justify-center', children=[
                                    enhance_location_switch,
                                    filter_critical_switch
                                ], style_='gap:4px; margin-right:8px;'),
                                v.Col(cols='auto', class_='d-flex flex-column justify-center', children=[
                                    (contact_activity_label := v.Html(tag='div', children=['Contacts with last activity: Last 24h'], style_='font-size:12px;color:#374151;margin:0 0 4px 0;line-height:1.2;')),
                                    (contact_activity_select := v.Select(items=['All','Last 1h','Last 24h','Last 7d','Custom…'], v_model='Last 24h', dense=True, filled=True, hide_details=True, class_='ma-0 pa-0', style_='min-width:180px;')),
                                    (contact_custom_range := v.Row(children=[
                                        (contact_from_picker := v.TextField(v_model='', label='From (YYYY-MM-DD HH:MM)', dense=True, hide_details=True, class_='ma-0 pa-0', style_='max-width:210px;')),
                                        (contact_to_picker := v.TextField(v_model='', label='To (YYYY-MM-DD HH:MM)', dense=True, hide_details=True, class_='ma-0 pa-0', style_='max-width:210px; margin-left:8px;'))
                                    ], class_='ma-0 pa-0')),
                                    (include_closed_checkbox := v.Checkbox(v_model=False, label='Include Closed', class_='mt-2'))
                                ], style_='gap:4px;')
                            ], align='center')
                        ]),
                        v.Spacer(),
                        v.Col(cols='auto', class_='d-flex align-center', children=[base_select], style_='margin-left:auto;')
                    ], align='center', class_='ict-toolbar', style_='display:flex; align-items:center;'),
                    m,
                ])
            )
        ])
    ]),
    v.TabItem(children=[
        v.Card(children=[
            v.CardTitle(children=['Log Table'], style_='border-top-left-radius:10px;border-top-right-radius:10px;'),
            v.CardText(style_='border-bottom-left-radius:10px;border-bottom-right-radius:10px;', children=[
                v.Row(children=[
                    widgets.HBox([edit_mode, create_btn], layout=widgets.Layout(margin='0')),
                    v.Spacer(),
                    widgets.HBox([prev_page_btn, page_size_dropdown, next_page_btn, page_label], layout=widgets.Layout(margin='0 10px')),
                    widgets.HBox([download_csv_btn, csv_download_link, update_map_btn], layout=widgets.Layout(margin='0'))
                ], align='center'),
                table_box
            ])
        ])
    ])
]
# Keep Tabs and TabsItems in sync (Voila needs explicit linkage)
def _tabs_changed(change):
    if change.get('name') == 'v_model':
        tab_items.v_model = change['new']
def _tabitems_changed(change):
    if change.get('name') == 'v_model':
        tabs.v_model = change['new']
tabs.observe(_tabs_changed, names=['v_model'])
tab_items.observe(_tabitems_changed, names=['v_model'])
vuetify_app = v.Container(children=[toolbar, tabs, tab_items], style_='max-width: 100vw;')
display(vuetify_app)
display(entry_dialog)
display(history_dialog)
display(delete_dialog)

ModuleNotFoundError: No module named 'ipywidgets'